
# COSC 6336 Natural Language Processing 
## Spring 2018
## In-class Assignment 2 - Language Models




### 1. (10) 
Write a function, `count_based_ngram(text, n)`, that takes as input a corpus, represented as a list of lists, and an integer value n, and generates a count-based n-gram language model. Do not worry about adding sentence delimiters.
Your function must return the parameters of the model in a dictionary.


In [5]:
from collections import defaultdict

def count_based_ngram(input_text, n):
    '''
    data: training data as a list of lists
    [[A, B, C, D],
     [D, B, E, A],
     :
     [A, B, D, D]
    ]
    
    n: size of n-gram
    
    returns:
    It must return the n-gram model parameters as a dictionary with the following structure 
    (for example, if n=2):

    { ('A', 'B') : 0.1,
      :
      ('D', 'B') : 0.2,
    }
    In this case the entry for ('A','B') means that P('B' | 'A') = 0.1
    In the same way, the entry for ('D', 'B') indicates that P(B | D ) = 0.2 and so on.
    The dictionary must not include conditional probabilities equal to 0, i.e. if some probability P(X | Y) = 0,
    there is not need for incluiding it in the dictionary.
    '''
    m = {}
    # your code here
    
    evid_count = defaultdict(lambda: 0)
    post_count = defaultdict(lambda: 0)

    for sentence in input_text:
        for i, word in enumerate(sentence):
            j = i + n
            t = tuple(sentence[i:j])
            
            if j <= len(sentence):
                evid_count[t[:-1]] += 1.
                post_count[t] += 1.
            elif j == len(sentence) + 1:
                evid_count[t] += 1.
            print()
    m = {}
    print(evid_count)
    print()
    print(post_count)
    for ngram, posterior in post_count.items():
        if posterior > 0 and evid_count[ngram[:-1]] > 0:
            m[ngram] = posterior / evid_count[ngram[:-1]]
    return m

In [6]:
input1 = [['A', 'B', 'C', 'D', 'E'],
          ['D', 'E', 'C', 'D', 'E'],
          ['A', 'C', 'D', 'D']]
input2 = [['<s>', 'I', 'am', 'Sam', '</s>'],
          ['<s>','Sam', 'I', 'am','</s>'],
          ['<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>']]

count_based_ngram(input1, 2)

('A', 'B')
--- 0 2 ('A', 'B')

('B', 'C')
--- 1 3 ('B', 'C')

('C', 'D')
--- 2 4 ('C', 'D')

('D', 'E')
--- 3 5 ('D', 'E')

('E',)
=== 4 6 ('E',)

('D', 'E')
--- 0 2 ('D', 'E')

('E', 'C')
--- 1 3 ('E', 'C')

('C', 'D')
--- 2 4 ('C', 'D')

('D', 'E')
--- 3 5 ('D', 'E')

('E',)
=== 4 6 ('E',)

('A', 'C')
--- 0 2 ('A', 'C')

('C', 'D')
--- 1 3 ('C', 'D')

('D', 'D')
--- 2 4 ('D', 'D')

('D',)
=== 3 5 ('D',)

defaultdict(<function count_based_ngram.<locals>.<lambda> at 0x108106b70>, {('A',): 2.0, ('B',): 1.0, ('C',): 3.0, ('D',): 5.0, ('E',): 3.0})

defaultdict(<function count_based_ngram.<locals>.<lambda> at 0x108106510>, {('A', 'B'): 1.0, ('B', 'C'): 1.0, ('C', 'D'): 3.0, ('D', 'E'): 3.0, ('E', 'C'): 1.0, ('A', 'C'): 1.0, ('D', 'D'): 1.0})


{('A', 'B'): 0.5,
 ('A', 'C'): 0.5,
 ('B', 'C'): 1.0,
 ('C', 'D'): 1.0,
 ('D', 'D'): 0.2,
 ('D', 'E'): 0.6,
 ('E', 'C'): 0.3333333333333333}

### 2. (20)
Extend your model using Add-1 (Laplace) smoothing by writing a function count_based_ngram_with_smoothing(text, n) that similar to the previous function, receives a corpus and the language model order as specified by n.
The function should return a dictionary with the model parameters in the same format as the previous model.

In [3]:
from itertools import product

def get_vocab(text):
     return set([word for sentence in text for word in sentence])

def count_based_ngram_with_smoothing(input_text, n):
    '''
    data: training data as a list of lists
    [[A, B, C, D],
     [D, E, E, A],
     :
     [A, B, D, D]
    ]

    n: size of n-gram

    returns:
    It must return the smoothed n-gram model parameters as a dictionary with the following structure 
    (for example, if n=2):

    { ('A', 'B') : 0.1,
      :
      ('D', 'B') : 0.2,
    }
    In this case the entry for ('A','B') means that P('B' | 'A') = 0.1
    In the same way, the entry for ('D', 'B') indicates that P(B | D ) = 0.2 and so on.
    '''
    vocab = get_vocab(input_text)
    ngrams = list(product(vocab, repeat=n))

    evid_count = {ngram[:-1]:0.0 for ngram in ngrams}
    post_count = {ngram:0.0 for ngram in ngrams}

    for sentence in input_text:
        for i in range(len(sentence)):
            j = i + n
            t = tuple(sentence[i:j])

            if j <= len(sentence):
                evid_count[t[:-1]] += 1.
                post_count[t] += 1.
            elif j == len(sentence) + 1:
                evid_count[t] += 1.
    m = {}
    for ngram, pcount in post_count.items():
        m[ngram] = (pcount + 1.0) / (evid_count[ngram[:-1]] + len(vocab) * 1.0)
    return m

In [4]:
count_based_ngram_with_smoothing(input1, 2)

{('A', 'A'): 0.14285714285714285,
 ('A', 'B'): 0.2857142857142857,
 ('A', 'C'): 0.2857142857142857,
 ('A', 'D'): 0.14285714285714285,
 ('A', 'E'): 0.14285714285714285,
 ('B', 'A'): 0.16666666666666666,
 ('B', 'B'): 0.16666666666666666,
 ('B', 'C'): 0.3333333333333333,
 ('B', 'D'): 0.16666666666666666,
 ('B', 'E'): 0.16666666666666666,
 ('C', 'A'): 0.125,
 ('C', 'B'): 0.125,
 ('C', 'C'): 0.125,
 ('C', 'D'): 0.5,
 ('C', 'E'): 0.125,
 ('D', 'A'): 0.1,
 ('D', 'B'): 0.1,
 ('D', 'C'): 0.1,
 ('D', 'D'): 0.2,
 ('D', 'E'): 0.4,
 ('E', 'A'): 0.125,
 ('E', 'B'): 0.125,
 ('E', 'C'): 0.25,
 ('E', 'D'): 0.125,
 ('E', 'E'): 0.125}

### 3. (20)
Add a function that uses your model to compute the probability of a given sequence. Use probablities and not log of probabilities. Your function receives as arguments the model, the sequence, and n, the order for the language model.

In [7]:
def word_ngrams(words, n):
    ngrams = []
    for offset in range(0, len(words) - n + 1):
        ngrams.append(tuple(words[offset:offset + n]))
    return ngrams

def prob_ngram(model, a, n):
    '''
    model: a dictionary with the model probabilities.
    a: a sequence of symbols (a sentence or fragment of the corpus)
    Returns: the probability P(a)
    '''
    # Your code here 
    prob = 1.0
    for ngram in word_ngrams(a, n):
        if ngram in model:
            prob *= model[tuple(ngram)]
        else:
            return 0
    return prob

## Grader
Run the following code to grade your exam

In [6]:
def approx_equal(val1, val2):
    return abs(val1-val2) <= 0.00001

def test_dict(test, answer):
    if sorted(test.keys()) != sorted(answer.keys()): return False
    for k,v in test.items():
        if not approx_equal(v,answer[k]):
            return False
    return True

input1 = [['A', 'B', 'C', 'D', 'E'],
     ['D', 'E', 'C', 'D', 'E'],
     ['A', 'C', 'D', 'D']
    ]
out1 = {('A', 'B'): 0.5, ('A', 'C'): 0.5, ('C', 'D'): 1.0, ('D', 'E'): 0.6, ('B', 'C'): 1.0, ('E', 'C'): 0.3333333333333333, ('D', 'D'): 0.2}

input2 = [['<s>', 'I', 'am', 'Sam', '</s>'],
        ['<s>','Sam', 'I', 'am','</s>'],
        ['<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>']]
out2 = {('I', 'am', 'Sam'): 0.5, ('I', 'do', 'not'): 1.0, ('do', 'not', 'like'): 1.0, ('Sam', 'I', 'am'): 1.0, ('<s>', 'I', 'am'): 0.5, ('eggs', 'and', 'ham'): 1.0, ('like', 'green', 'eggs'): 1.0, ('<s>', 'I', 'do'): 0.5, ('and', 'ham', '</s>'): 1.0, ('<s>', 'Sam', 'I'): 1.0, ('I', 'am', '</s>'): 0.5, ('am', 'Sam', '</s>'): 1.0, ('not', 'like', 'green'): 1.0, ('green', 'eggs', 'and'): 1.0}

def test1():
    student = count_based_ngram(input1, 2)
    if not test_dict(out1, student):
        return False
    student = count_based_ngram(input2, 3)
    if not test_dict(out2, student):
        return False
    return True

smooth_out1 = {('A', 'B'): 0.2857142857142857, ('B', 'C'): 0.3333333333333333, ('C', 'D'): 0.5, ('D', 'E'): 0.4, ('E', 'C'): 0.25, ('A', 'C'): 0.2857142857142857, ('D', 'D'): 0.2, ('A', 'A'): 0.14285714285714285, ('A', 'D'): 0.14285714285714285, ('A', 'E'): 0.14285714285714285, ('B', 'A'): 0.16666666666666666, ('B', 'B'): 0.16666666666666666, ('B', 'D'): 0.16666666666666666, ('B', 'E'): 0.16666666666666666, ('C', 'A'): 0.125, ('C', 'B'): 0.125, ('C', 'C'): 0.125, ('C', 'E'): 0.125, ('D', 'A'): 0.1, ('D', 'B'): 0.1, ('D', 'C'): 0.1, ('E', 'A'): 0.125, ('E', 'B'): 0.125, ('E', 'D'): 0.125, ('E', 'E'): 0.125}
smooth_out2 = {('<s>', 'I', 'am'): 0.14285714285714285, ('I', 'am', 'Sam'): 0.14285714285714285, ('am', 'Sam', '</s>'): 0.15384615384615385, ('<s>', 'Sam', 'I'): 0.15384615384615385, ('Sam', 'I', 'am'): 0.15384615384615385, ('I', 'am', '</s>'): 0.14285714285714285, ('<s>', 'I', 'do'): 0.14285714285714285, ('I', 'do', 'not'): 0.15384615384615385, ('do', 'not', 'like'): 0.15384615384615385, ('not', 'like', 'green'): 0.15384615384615385, ('like', 'green', 'eggs'): 0.15384615384615385, ('green', 'eggs', 'and'): 0.15384615384615385, ('eggs', 'and', 'ham'): 0.15384615384615385, ('and', 'ham', '</s>'): 0.15384615384615385, ('<s>', '<s>', '<s>'): 0.08333333333333333, ('<s>', '<s>', 'I'): 0.08333333333333333, ('<s>', '<s>', 'am'): 0.08333333333333333, ('<s>', '<s>', 'Sam'): 0.08333333333333333, ('<s>', '<s>', '</s>'): 0.08333333333333333, ('<s>', '<s>', 'do'): 0.08333333333333333, ('<s>', '<s>', 'not'): 0.08333333333333333, ('<s>', '<s>', 'like'): 0.08333333333333333, ('<s>', '<s>', 'green'): 0.08333333333333333, ('<s>', '<s>', 'eggs'): 0.08333333333333333, ('<s>', '<s>', 'and'): 0.08333333333333333, ('<s>', '<s>', 'ham'): 0.08333333333333333, ('<s>', 'I', '<s>'): 0.07142857142857142, ('<s>', 'I', 'I'): 0.07142857142857142, ('<s>', 'I', 'Sam'): 0.07142857142857142, ('<s>', 'I', '</s>'): 0.07142857142857142, ('<s>', 'I', 'not'): 0.07142857142857142, ('<s>', 'I', 'like'): 0.07142857142857142, ('<s>', 'I', 'green'): 0.07142857142857142, ('<s>', 'I', 'eggs'): 0.07142857142857142, ('<s>', 'I', 'and'): 0.07142857142857142, ('<s>', 'I', 'ham'): 0.07142857142857142, ('<s>', 'am', '<s>'): 0.08333333333333333, ('<s>', 'am', 'I'): 0.08333333333333333, ('<s>', 'am', 'am'): 0.08333333333333333, ('<s>', 'am', 'Sam'): 0.08333333333333333, ('<s>', 'am', '</s>'): 0.08333333333333333, ('<s>', 'am', 'do'): 0.08333333333333333, ('<s>', 'am', 'not'): 0.08333333333333333, ('<s>', 'am', 'like'): 0.08333333333333333, ('<s>', 'am', 'green'): 0.08333333333333333, ('<s>', 'am', 'eggs'): 0.08333333333333333, ('<s>', 'am', 'and'): 0.08333333333333333, ('<s>', 'am', 'ham'): 0.08333333333333333, ('<s>', 'Sam', '<s>'): 0.07692307692307693, ('<s>', 'Sam', 'am'): 0.07692307692307693, ('<s>', 'Sam', 'Sam'): 0.07692307692307693, ('<s>', 'Sam', '</s>'): 0.07692307692307693, ('<s>', 'Sam', 'do'): 0.07692307692307693, ('<s>', 'Sam', 'not'): 0.07692307692307693, ('<s>', 'Sam', 'like'): 0.07692307692307693, ('<s>', 'Sam', 'green'): 0.07692307692307693, ('<s>', 'Sam', 'eggs'): 0.07692307692307693, ('<s>', 'Sam', 'and'): 0.07692307692307693, ('<s>', 'Sam', 'ham'): 0.07692307692307693, ('<s>', '</s>', '<s>'): 0.08333333333333333, ('<s>', '</s>', 'I'): 0.08333333333333333, ('<s>', '</s>', 'am'): 0.08333333333333333, ('<s>', '</s>', 'Sam'): 0.08333333333333333, ('<s>', '</s>', '</s>'): 0.08333333333333333, ('<s>', '</s>', 'do'): 0.08333333333333333, ('<s>', '</s>', 'not'): 0.08333333333333333, ('<s>', '</s>', 'like'): 0.08333333333333333, ('<s>', '</s>', 'green'): 0.08333333333333333, ('<s>', '</s>', 'eggs'): 0.08333333333333333, ('<s>', '</s>', 'and'): 0.08333333333333333, ('<s>', '</s>', 'ham'): 0.08333333333333333, ('<s>', 'do', '<s>'): 0.08333333333333333, ('<s>', 'do', 'I'): 0.08333333333333333, ('<s>', 'do', 'am'): 0.08333333333333333, ('<s>', 'do', 'Sam'): 0.08333333333333333, ('<s>', 'do', '</s>'): 0.08333333333333333, ('<s>', 'do', 'do'): 0.08333333333333333, ('<s>', 'do', 'not'): 0.08333333333333333, ('<s>', 'do', 'like'): 0.08333333333333333, ('<s>', 'do', 'green'): 0.08333333333333333, ('<s>', 'do', 'eggs'): 0.08333333333333333, ('<s>', 'do', 'and'): 0.08333333333333333, ('<s>', 'do', 'ham'): 0.08333333333333333, ('<s>', 'not', '<s>'): 0.08333333333333333, ('<s>', 'not', 'I'): 0.08333333333333333, ('<s>', 'not', 'am'): 0.08333333333333333, ('<s>', 'not', 'Sam'): 0.08333333333333333, ('<s>', 'not', '</s>'): 0.08333333333333333, ('<s>', 'not', 'do'): 0.08333333333333333, ('<s>', 'not', 'not'): 0.08333333333333333, ('<s>', 'not', 'like'): 0.08333333333333333, ('<s>', 'not', 'green'): 0.08333333333333333, ('<s>', 'not', 'eggs'): 0.08333333333333333, ('<s>', 'not', 'and'): 0.08333333333333333, ('<s>', 'not', 'ham'): 0.08333333333333333, ('<s>', 'like', '<s>'): 0.08333333333333333, ('<s>', 'like', 'I'): 0.08333333333333333, ('<s>', 'like', 'am'): 0.08333333333333333, ('<s>', 'like', 'Sam'): 0.08333333333333333, ('<s>', 'like', '</s>'): 0.08333333333333333, ('<s>', 'like', 'do'): 0.08333333333333333, ('<s>', 'like', 'not'): 0.08333333333333333, ('<s>', 'like', 'like'): 0.08333333333333333, ('<s>', 'like', 'green'): 0.08333333333333333, ('<s>', 'like', 'eggs'): 0.08333333333333333, ('<s>', 'like', 'and'): 0.08333333333333333, ('<s>', 'like', 'ham'): 0.08333333333333333, ('<s>', 'green', '<s>'): 0.08333333333333333, ('<s>', 'green', 'I'): 0.08333333333333333, ('<s>', 'green', 'am'): 0.08333333333333333, ('<s>', 'green', 'Sam'): 0.08333333333333333, ('<s>', 'green', '</s>'): 0.08333333333333333, ('<s>', 'green', 'do'): 0.08333333333333333, ('<s>', 'green', 'not'): 0.08333333333333333, ('<s>', 'green', 'like'): 0.08333333333333333, ('<s>', 'green', 'green'): 0.08333333333333333, ('<s>', 'green', 'eggs'): 0.08333333333333333, ('<s>', 'green', 'and'): 0.08333333333333333, ('<s>', 'green', 'ham'): 0.08333333333333333, ('<s>', 'eggs', '<s>'): 0.08333333333333333, ('<s>', 'eggs', 'I'): 0.08333333333333333, ('<s>', 'eggs', 'am'): 0.08333333333333333, ('<s>', 'eggs', 'Sam'): 0.08333333333333333, ('<s>', 'eggs', '</s>'): 0.08333333333333333, ('<s>', 'eggs', 'do'): 0.08333333333333333, ('<s>', 'eggs', 'not'): 0.08333333333333333, ('<s>', 'eggs', 'like'): 0.08333333333333333, ('<s>', 'eggs', 'green'): 0.08333333333333333, ('<s>', 'eggs', 'eggs'): 0.08333333333333333, ('<s>', 'eggs', 'and'): 0.08333333333333333, ('<s>', 'eggs', 'ham'): 0.08333333333333333, ('<s>', 'and', '<s>'): 0.08333333333333333, ('<s>', 'and', 'I'): 0.08333333333333333, ('<s>', 'and', 'am'): 0.08333333333333333, ('<s>', 'and', 'Sam'): 0.08333333333333333, ('<s>', 'and', '</s>'): 0.08333333333333333, ('<s>', 'and', 'do'): 0.08333333333333333, ('<s>', 'and', 'not'): 0.08333333333333333, ('<s>', 'and', 'like'): 0.08333333333333333, ('<s>', 'and', 'green'): 0.08333333333333333, ('<s>', 'and', 'eggs'): 0.08333333333333333, ('<s>', 'and', 'and'): 0.08333333333333333, ('<s>', 'and', 'ham'): 0.08333333333333333, ('<s>', 'ham', '<s>'): 0.08333333333333333, ('<s>', 'ham', 'I'): 0.08333333333333333, ('<s>', 'ham', 'am'): 0.08333333333333333, ('<s>', 'ham', 'Sam'): 0.08333333333333333, ('<s>', 'ham', '</s>'): 0.08333333333333333, ('<s>', 'ham', 'do'): 0.08333333333333333, ('<s>', 'ham', 'not'): 0.08333333333333333, ('<s>', 'ham', 'like'): 0.08333333333333333, ('<s>', 'ham', 'green'): 0.08333333333333333, ('<s>', 'ham', 'eggs'): 0.08333333333333333, ('<s>', 'ham', 'and'): 0.08333333333333333, ('<s>', 'ham', 'ham'): 0.08333333333333333, ('I', '<s>', '<s>'): 0.08333333333333333, ('I', '<s>', 'I'): 0.08333333333333333, ('I', '<s>', 'am'): 0.08333333333333333, ('I', '<s>', 'Sam'): 0.08333333333333333, ('I', '<s>', '</s>'): 0.08333333333333333, ('I', '<s>', 'do'): 0.08333333333333333, ('I', '<s>', 'not'): 0.08333333333333333, ('I', '<s>', 'like'): 0.08333333333333333, ('I', '<s>', 'green'): 0.08333333333333333, ('I', '<s>', 'eggs'): 0.08333333333333333, ('I', '<s>', 'and'): 0.08333333333333333, ('I', '<s>', 'ham'): 0.08333333333333333, ('I', 'I', '<s>'): 0.08333333333333333, ('I', 'I', 'I'): 0.08333333333333333, ('I', 'I', 'am'): 0.08333333333333333, ('I', 'I', 'Sam'): 0.08333333333333333, ('I', 'I', '</s>'): 0.08333333333333333, ('I', 'I', 'do'): 0.08333333333333333, ('I', 'I', 'not'): 0.08333333333333333, ('I', 'I', 'like'): 0.08333333333333333, ('I', 'I', 'green'): 0.08333333333333333, ('I', 'I', 'eggs'): 0.08333333333333333, ('I', 'I', 'and'): 0.08333333333333333, ('I', 'I', 'ham'): 0.08333333333333333, ('I', 'am', '<s>'): 0.07142857142857142, ('I', 'am', 'I'): 0.07142857142857142, ('I', 'am', 'am'): 0.07142857142857142, ('I', 'am', 'do'): 0.07142857142857142, ('I', 'am', 'not'): 0.07142857142857142, ('I', 'am', 'like'): 0.07142857142857142, ('I', 'am', 'green'): 0.07142857142857142, ('I', 'am', 'eggs'): 0.07142857142857142, ('I', 'am', 'and'): 0.07142857142857142, ('I', 'am', 'ham'): 0.07142857142857142, ('I', 'Sam', '<s>'): 0.08333333333333333, ('I', 'Sam', 'I'): 0.08333333333333333, ('I', 'Sam', 'am'): 0.08333333333333333, ('I', 'Sam', 'Sam'): 0.08333333333333333, ('I', 'Sam', '</s>'): 0.08333333333333333, ('I', 'Sam', 'do'): 0.08333333333333333, ('I', 'Sam', 'not'): 0.08333333333333333, ('I', 'Sam', 'like'): 0.08333333333333333, ('I', 'Sam', 'green'): 0.08333333333333333, ('I', 'Sam', 'eggs'): 0.08333333333333333, ('I', 'Sam', 'and'): 0.08333333333333333, ('I', 'Sam', 'ham'): 0.08333333333333333, ('I', '</s>', '<s>'): 0.08333333333333333, ('I', '</s>', 'I'): 0.08333333333333333, ('I', '</s>', 'am'): 0.08333333333333333, ('I', '</s>', 'Sam'): 0.08333333333333333, ('I', '</s>', '</s>'): 0.08333333333333333, ('I', '</s>', 'do'): 0.08333333333333333, ('I', '</s>', 'not'): 0.08333333333333333, ('I', '</s>', 'like'): 0.08333333333333333, ('I', '</s>', 'green'): 0.08333333333333333, ('I', '</s>', 'eggs'): 0.08333333333333333, ('I', '</s>', 'and'): 0.08333333333333333, ('I', '</s>', 'ham'): 0.08333333333333333, ('I', 'do', '<s>'): 0.07692307692307693, ('I', 'do', 'I'): 0.07692307692307693, ('I', 'do', 'am'): 0.07692307692307693, ('I', 'do', 'Sam'): 0.07692307692307693, ('I', 'do', '</s>'): 0.07692307692307693, ('I', 'do', 'do'): 0.07692307692307693, ('I', 'do', 'like'): 0.07692307692307693, ('I', 'do', 'green'): 0.07692307692307693, ('I', 'do', 'eggs'): 0.07692307692307693, ('I', 'do', 'and'): 0.07692307692307693, ('I', 'do', 'ham'): 0.07692307692307693, ('I', 'not', '<s>'): 0.08333333333333333, ('I', 'not', 'I'): 0.08333333333333333, ('I', 'not', 'am'): 0.08333333333333333, ('I', 'not', 'Sam'): 0.08333333333333333, ('I', 'not', '</s>'): 0.08333333333333333, ('I', 'not', 'do'): 0.08333333333333333, ('I', 'not', 'not'): 0.08333333333333333, ('I', 'not', 'like'): 0.08333333333333333, ('I', 'not', 'green'): 0.08333333333333333, ('I', 'not', 'eggs'): 0.08333333333333333, ('I', 'not', 'and'): 0.08333333333333333, ('I', 'not', 'ham'): 0.08333333333333333, ('I', 'like', '<s>'): 0.08333333333333333, ('I', 'like', 'I'): 0.08333333333333333, ('I', 'like', 'am'): 0.08333333333333333, ('I', 'like', 'Sam'): 0.08333333333333333, ('I', 'like', '</s>'): 0.08333333333333333, ('I', 'like', 'do'): 0.08333333333333333, ('I', 'like', 'not'): 0.08333333333333333, ('I', 'like', 'like'): 0.08333333333333333, ('I', 'like', 'green'): 0.08333333333333333, ('I', 'like', 'eggs'): 0.08333333333333333, ('I', 'like', 'and'): 0.08333333333333333, ('I', 'like', 'ham'): 0.08333333333333333, ('I', 'green', '<s>'): 0.08333333333333333, ('I', 'green', 'I'): 0.08333333333333333, ('I', 'green', 'am'): 0.08333333333333333, ('I', 'green', 'Sam'): 0.08333333333333333, ('I', 'green', '</s>'): 0.08333333333333333, ('I', 'green', 'do'): 0.08333333333333333, ('I', 'green', 'not'): 0.08333333333333333, ('I', 'green', 'like'): 0.08333333333333333, ('I', 'green', 'green'): 0.08333333333333333, ('I', 'green', 'eggs'): 0.08333333333333333, ('I', 'green', 'and'): 0.08333333333333333, ('I', 'green', 'ham'): 0.08333333333333333, ('I', 'eggs', '<s>'): 0.08333333333333333, ('I', 'eggs', 'I'): 0.08333333333333333, ('I', 'eggs', 'am'): 0.08333333333333333, ('I', 'eggs', 'Sam'): 0.08333333333333333, ('I', 'eggs', '</s>'): 0.08333333333333333, ('I', 'eggs', 'do'): 0.08333333333333333, ('I', 'eggs', 'not'): 0.08333333333333333, ('I', 'eggs', 'like'): 0.08333333333333333, ('I', 'eggs', 'green'): 0.08333333333333333, ('I', 'eggs', 'eggs'): 0.08333333333333333, ('I', 'eggs', 'and'): 0.08333333333333333, ('I', 'eggs', 'ham'): 0.08333333333333333, ('I', 'and', '<s>'): 0.08333333333333333, ('I', 'and', 'I'): 0.08333333333333333, ('I', 'and', 'am'): 0.08333333333333333, ('I', 'and', 'Sam'): 0.08333333333333333, ('I', 'and', '</s>'): 0.08333333333333333, ('I', 'and', 'do'): 0.08333333333333333, ('I', 'and', 'not'): 0.08333333333333333, ('I', 'and', 'like'): 0.08333333333333333, ('I', 'and', 'green'): 0.08333333333333333, ('I', 'and', 'eggs'): 0.08333333333333333, ('I', 'and', 'and'): 0.08333333333333333, ('I', 'and', 'ham'): 0.08333333333333333, ('I', 'ham', '<s>'): 0.08333333333333333, ('I', 'ham', 'I'): 0.08333333333333333, ('I', 'ham', 'am'): 0.08333333333333333, ('I', 'ham', 'Sam'): 0.08333333333333333, ('I', 'ham', '</s>'): 0.08333333333333333, ('I', 'ham', 'do'): 0.08333333333333333, ('I', 'ham', 'not'): 0.08333333333333333, ('I', 'ham', 'like'): 0.08333333333333333, ('I', 'ham', 'green'): 0.08333333333333333, ('I', 'ham', 'eggs'): 0.08333333333333333, ('I', 'ham', 'and'): 0.08333333333333333, ('I', 'ham', 'ham'): 0.08333333333333333, ('am', '<s>', '<s>'): 0.08333333333333333, ('am', '<s>', 'I'): 0.08333333333333333, ('am', '<s>', 'am'): 0.08333333333333333, ('am', '<s>', 'Sam'): 0.08333333333333333, ('am', '<s>', '</s>'): 0.08333333333333333, ('am', '<s>', 'do'): 0.08333333333333333, ('am', '<s>', 'not'): 0.08333333333333333, ('am', '<s>', 'like'): 0.08333333333333333, ('am', '<s>', 'green'): 0.08333333333333333, ('am', '<s>', 'eggs'): 0.08333333333333333, ('am', '<s>', 'and'): 0.08333333333333333, ('am', '<s>', 'ham'): 0.08333333333333333, ('am', 'I', '<s>'): 0.08333333333333333, ('am', 'I', 'I'): 0.08333333333333333, ('am', 'I', 'am'): 0.08333333333333333, ('am', 'I', 'Sam'): 0.08333333333333333, ('am', 'I', '</s>'): 0.08333333333333333, ('am', 'I', 'do'): 0.08333333333333333, ('am', 'I', 'not'): 0.08333333333333333, ('am', 'I', 'like'): 0.08333333333333333, ('am', 'I', 'green'): 0.08333333333333333, ('am', 'I', 'eggs'): 0.08333333333333333, ('am', 'I', 'and'): 0.08333333333333333, ('am', 'I', 'ham'): 0.08333333333333333, ('am', 'am', '<s>'): 0.08333333333333333, ('am', 'am', 'I'): 0.08333333333333333, ('am', 'am', 'am'): 0.08333333333333333, ('am', 'am', 'Sam'): 0.08333333333333333, ('am', 'am', '</s>'): 0.08333333333333333, ('am', 'am', 'do'): 0.08333333333333333, ('am', 'am', 'not'): 0.08333333333333333, ('am', 'am', 'like'): 0.08333333333333333, ('am', 'am', 'green'): 0.08333333333333333, ('am', 'am', 'eggs'): 0.08333333333333333, ('am', 'am', 'and'): 0.08333333333333333, ('am', 'am', 'ham'): 0.08333333333333333, ('am', 'Sam', '<s>'): 0.07692307692307693, ('am', 'Sam', 'I'): 0.07692307692307693, ('am', 'Sam', 'am'): 0.07692307692307693, ('am', 'Sam', 'Sam'): 0.07692307692307693, ('am', 'Sam', 'do'): 0.07692307692307693, ('am', 'Sam', 'not'): 0.07692307692307693, ('am', 'Sam', 'like'): 0.07692307692307693, ('am', 'Sam', 'green'): 0.07692307692307693, ('am', 'Sam', 'eggs'): 0.07692307692307693, ('am', 'Sam', 'and'): 0.07692307692307693, ('am', 'Sam', 'ham'): 0.07692307692307693, ('am', '</s>', '<s>'): 0.07692307692307693, ('am', '</s>', 'I'): 0.07692307692307693, ('am', '</s>', 'am'): 0.07692307692307693, ('am', '</s>', 'Sam'): 0.07692307692307693, ('am', '</s>', '</s>'): 0.07692307692307693, ('am', '</s>', 'do'): 0.07692307692307693, ('am', '</s>', 'not'): 0.07692307692307693, ('am', '</s>', 'like'): 0.07692307692307693, ('am', '</s>', 'green'): 0.07692307692307693, ('am', '</s>', 'eggs'): 0.07692307692307693, ('am', '</s>', 'and'): 0.07692307692307693, ('am', '</s>', 'ham'): 0.07692307692307693, ('am', 'do', '<s>'): 0.08333333333333333, ('am', 'do', 'I'): 0.08333333333333333, ('am', 'do', 'am'): 0.08333333333333333, ('am', 'do', 'Sam'): 0.08333333333333333, ('am', 'do', '</s>'): 0.08333333333333333, ('am', 'do', 'do'): 0.08333333333333333, ('am', 'do', 'not'): 0.08333333333333333, ('am', 'do', 'like'): 0.08333333333333333, ('am', 'do', 'green'): 0.08333333333333333, ('am', 'do', 'eggs'): 0.08333333333333333, ('am', 'do', 'and'): 0.08333333333333333, ('am', 'do', 'ham'): 0.08333333333333333, ('am', 'not', '<s>'): 0.08333333333333333, ('am', 'not', 'I'): 0.08333333333333333, ('am', 'not', 'am'): 0.08333333333333333, ('am', 'not', 'Sam'): 0.08333333333333333, ('am', 'not', '</s>'): 0.08333333333333333, ('am', 'not', 'do'): 0.08333333333333333, ('am', 'not', 'not'): 0.08333333333333333, ('am', 'not', 'like'): 0.08333333333333333, ('am', 'not', 'green'): 0.08333333333333333, ('am', 'not', 'eggs'): 0.08333333333333333, ('am', 'not', 'and'): 0.08333333333333333, ('am', 'not', 'ham'): 0.08333333333333333, ('am', 'like', '<s>'): 0.08333333333333333, ('am', 'like', 'I'): 0.08333333333333333, ('am', 'like', 'am'): 0.08333333333333333, ('am', 'like', 'Sam'): 0.08333333333333333, ('am', 'like', '</s>'): 0.08333333333333333, ('am', 'like', 'do'): 0.08333333333333333, ('am', 'like', 'not'): 0.08333333333333333, ('am', 'like', 'like'): 0.08333333333333333, ('am', 'like', 'green'): 0.08333333333333333, ('am', 'like', 'eggs'): 0.08333333333333333, ('am', 'like', 'and'): 0.08333333333333333, ('am', 'like', 'ham'): 0.08333333333333333, ('am', 'green', '<s>'): 0.08333333333333333, ('am', 'green', 'I'): 0.08333333333333333, ('am', 'green', 'am'): 0.08333333333333333, ('am', 'green', 'Sam'): 0.08333333333333333, ('am', 'green', '</s>'): 0.08333333333333333, ('am', 'green', 'do'): 0.08333333333333333, ('am', 'green', 'not'): 0.08333333333333333, ('am', 'green', 'like'): 0.08333333333333333, ('am', 'green', 'green'): 0.08333333333333333, ('am', 'green', 'eggs'): 0.08333333333333333, ('am', 'green', 'and'): 0.08333333333333333, ('am', 'green', 'ham'): 0.08333333333333333, ('am', 'eggs', '<s>'): 0.08333333333333333, ('am', 'eggs', 'I'): 0.08333333333333333, ('am', 'eggs', 'am'): 0.08333333333333333, ('am', 'eggs', 'Sam'): 0.08333333333333333, ('am', 'eggs', '</s>'): 0.08333333333333333, ('am', 'eggs', 'do'): 0.08333333333333333, ('am', 'eggs', 'not'): 0.08333333333333333, ('am', 'eggs', 'like'): 0.08333333333333333, ('am', 'eggs', 'green'): 0.08333333333333333, ('am', 'eggs', 'eggs'): 0.08333333333333333, ('am', 'eggs', 'and'): 0.08333333333333333, ('am', 'eggs', 'ham'): 0.08333333333333333, ('am', 'and', '<s>'): 0.08333333333333333, ('am', 'and', 'I'): 0.08333333333333333, ('am', 'and', 'am'): 0.08333333333333333, ('am', 'and', 'Sam'): 0.08333333333333333, ('am', 'and', '</s>'): 0.08333333333333333, ('am', 'and', 'do'): 0.08333333333333333, ('am', 'and', 'not'): 0.08333333333333333, ('am', 'and', 'like'): 0.08333333333333333, ('am', 'and', 'green'): 0.08333333333333333, ('am', 'and', 'eggs'): 0.08333333333333333, ('am', 'and', 'and'): 0.08333333333333333, ('am', 'and', 'ham'): 0.08333333333333333, ('am', 'ham', '<s>'): 0.08333333333333333, ('am', 'ham', 'I'): 0.08333333333333333, ('am', 'ham', 'am'): 0.08333333333333333, ('am', 'ham', 'Sam'): 0.08333333333333333, ('am', 'ham', '</s>'): 0.08333333333333333, ('am', 'ham', 'do'): 0.08333333333333333, ('am', 'ham', 'not'): 0.08333333333333333, ('am', 'ham', 'like'): 0.08333333333333333, ('am', 'ham', 'green'): 0.08333333333333333, ('am', 'ham', 'eggs'): 0.08333333333333333, ('am', 'ham', 'and'): 0.08333333333333333, ('am', 'ham', 'ham'): 0.08333333333333333, ('Sam', '<s>', '<s>'): 0.08333333333333333, ('Sam', '<s>', 'I'): 0.08333333333333333, ('Sam', '<s>', 'am'): 0.08333333333333333, ('Sam', '<s>', 'Sam'): 0.08333333333333333, ('Sam', '<s>', '</s>'): 0.08333333333333333, ('Sam', '<s>', 'do'): 0.08333333333333333, ('Sam', '<s>', 'not'): 0.08333333333333333, ('Sam', '<s>', 'like'): 0.08333333333333333, ('Sam', '<s>', 'green'): 0.08333333333333333, ('Sam', '<s>', 'eggs'): 0.08333333333333333, ('Sam', '<s>', 'and'): 0.08333333333333333, ('Sam', '<s>', 'ham'): 0.08333333333333333, ('Sam', 'I', '<s>'): 0.07692307692307693, ('Sam', 'I', 'I'): 0.07692307692307693, ('Sam', 'I', 'Sam'): 0.07692307692307693, ('Sam', 'I', '</s>'): 0.07692307692307693, ('Sam', 'I', 'do'): 0.07692307692307693, ('Sam', 'I', 'not'): 0.07692307692307693, ('Sam', 'I', 'like'): 0.07692307692307693, ('Sam', 'I', 'green'): 0.07692307692307693, ('Sam', 'I', 'eggs'): 0.07692307692307693, ('Sam', 'I', 'and'): 0.07692307692307693, ('Sam', 'I', 'ham'): 0.07692307692307693, ('Sam', 'am', '<s>'): 0.08333333333333333, ('Sam', 'am', 'I'): 0.08333333333333333, ('Sam', 'am', 'am'): 0.08333333333333333, ('Sam', 'am', 'Sam'): 0.08333333333333333, ('Sam', 'am', '</s>'): 0.08333333333333333, ('Sam', 'am', 'do'): 0.08333333333333333, ('Sam', 'am', 'not'): 0.08333333333333333, ('Sam', 'am', 'like'): 0.08333333333333333, ('Sam', 'am', 'green'): 0.08333333333333333, ('Sam', 'am', 'eggs'): 0.08333333333333333, ('Sam', 'am', 'and'): 0.08333333333333333, ('Sam', 'am', 'ham'): 0.08333333333333333, ('Sam', 'Sam', '<s>'): 0.08333333333333333, ('Sam', 'Sam', 'I'): 0.08333333333333333, ('Sam', 'Sam', 'am'): 0.08333333333333333, ('Sam', 'Sam', 'Sam'): 0.08333333333333333, ('Sam', 'Sam', '</s>'): 0.08333333333333333, ('Sam', 'Sam', 'do'): 0.08333333333333333, ('Sam', 'Sam', 'not'): 0.08333333333333333, ('Sam', 'Sam', 'like'): 0.08333333333333333, ('Sam', 'Sam', 'green'): 0.08333333333333333, ('Sam', 'Sam', 'eggs'): 0.08333333333333333, ('Sam', 'Sam', 'and'): 0.08333333333333333, ('Sam', 'Sam', 'ham'): 0.08333333333333333, ('Sam', '</s>', '<s>'): 0.07692307692307693, ('Sam', '</s>', 'I'): 0.07692307692307693, ('Sam', '</s>', 'am'): 0.07692307692307693, ('Sam', '</s>', 'Sam'): 0.07692307692307693, ('Sam', '</s>', '</s>'): 0.07692307692307693, ('Sam', '</s>', 'do'): 0.07692307692307693, ('Sam', '</s>', 'not'): 0.07692307692307693, ('Sam', '</s>', 'like'): 0.07692307692307693, ('Sam', '</s>', 'green'): 0.07692307692307693, ('Sam', '</s>', 'eggs'): 0.07692307692307693, ('Sam', '</s>', 'and'): 0.07692307692307693, ('Sam', '</s>', 'ham'): 0.07692307692307693, ('Sam', 'do', '<s>'): 0.08333333333333333, ('Sam', 'do', 'I'): 0.08333333333333333, ('Sam', 'do', 'am'): 0.08333333333333333, ('Sam', 'do', 'Sam'): 0.08333333333333333, ('Sam', 'do', '</s>'): 0.08333333333333333, ('Sam', 'do', 'do'): 0.08333333333333333, ('Sam', 'do', 'not'): 0.08333333333333333, ('Sam', 'do', 'like'): 0.08333333333333333, ('Sam', 'do', 'green'): 0.08333333333333333, ('Sam', 'do', 'eggs'): 0.08333333333333333, ('Sam', 'do', 'and'): 0.08333333333333333, ('Sam', 'do', 'ham'): 0.08333333333333333, ('Sam', 'not', '<s>'): 0.08333333333333333, ('Sam', 'not', 'I'): 0.08333333333333333, ('Sam', 'not', 'am'): 0.08333333333333333, ('Sam', 'not', 'Sam'): 0.08333333333333333, ('Sam', 'not', '</s>'): 0.08333333333333333, ('Sam', 'not', 'do'): 0.08333333333333333, ('Sam', 'not', 'not'): 0.08333333333333333, ('Sam', 'not', 'like'): 0.08333333333333333, ('Sam', 'not', 'green'): 0.08333333333333333, ('Sam', 'not', 'eggs'): 0.08333333333333333, ('Sam', 'not', 'and'): 0.08333333333333333, ('Sam', 'not', 'ham'): 0.08333333333333333, ('Sam', 'like', '<s>'): 0.08333333333333333, ('Sam', 'like', 'I'): 0.08333333333333333, ('Sam', 'like', 'am'): 0.08333333333333333, ('Sam', 'like', 'Sam'): 0.08333333333333333, ('Sam', 'like', '</s>'): 0.08333333333333333, ('Sam', 'like', 'do'): 0.08333333333333333, ('Sam', 'like', 'not'): 0.08333333333333333, ('Sam', 'like', 'like'): 0.08333333333333333, ('Sam', 'like', 'green'): 0.08333333333333333, ('Sam', 'like', 'eggs'): 0.08333333333333333, ('Sam', 'like', 'and'): 0.08333333333333333, ('Sam', 'like', 'ham'): 0.08333333333333333, ('Sam', 'green', '<s>'): 0.08333333333333333, ('Sam', 'green', 'I'): 0.08333333333333333, ('Sam', 'green', 'am'): 0.08333333333333333, ('Sam', 'green', 'Sam'): 0.08333333333333333, ('Sam', 'green', '</s>'): 0.08333333333333333, ('Sam', 'green', 'do'): 0.08333333333333333, ('Sam', 'green', 'not'): 0.08333333333333333, ('Sam', 'green', 'like'): 0.08333333333333333, ('Sam', 'green', 'green'): 0.08333333333333333, ('Sam', 'green', 'eggs'): 0.08333333333333333, ('Sam', 'green', 'and'): 0.08333333333333333, ('Sam', 'green', 'ham'): 0.08333333333333333, ('Sam', 'eggs', '<s>'): 0.08333333333333333, ('Sam', 'eggs', 'I'): 0.08333333333333333, ('Sam', 'eggs', 'am'): 0.08333333333333333, ('Sam', 'eggs', 'Sam'): 0.08333333333333333, ('Sam', 'eggs', '</s>'): 0.08333333333333333, ('Sam', 'eggs', 'do'): 0.08333333333333333, ('Sam', 'eggs', 'not'): 0.08333333333333333, ('Sam', 'eggs', 'like'): 0.08333333333333333, ('Sam', 'eggs', 'green'): 0.08333333333333333, ('Sam', 'eggs', 'eggs'): 0.08333333333333333, ('Sam', 'eggs', 'and'): 0.08333333333333333, ('Sam', 'eggs', 'ham'): 0.08333333333333333, ('Sam', 'and', '<s>'): 0.08333333333333333, ('Sam', 'and', 'I'): 0.08333333333333333, ('Sam', 'and', 'am'): 0.08333333333333333, ('Sam', 'and', 'Sam'): 0.08333333333333333, ('Sam', 'and', '</s>'): 0.08333333333333333, ('Sam', 'and', 'do'): 0.08333333333333333, ('Sam', 'and', 'not'): 0.08333333333333333, ('Sam', 'and', 'like'): 0.08333333333333333, ('Sam', 'and', 'green'): 0.08333333333333333, ('Sam', 'and', 'eggs'): 0.08333333333333333, ('Sam', 'and', 'and'): 0.08333333333333333, ('Sam', 'and', 'ham'): 0.08333333333333333, ('Sam', 'ham', '<s>'): 0.08333333333333333, ('Sam', 'ham', 'I'): 0.08333333333333333, ('Sam', 'ham', 'am'): 0.08333333333333333, ('Sam', 'ham', 'Sam'): 0.08333333333333333, ('Sam', 'ham', '</s>'): 0.08333333333333333, ('Sam', 'ham', 'do'): 0.08333333333333333, ('Sam', 'ham', 'not'): 0.08333333333333333, ('Sam', 'ham', 'like'): 0.08333333333333333, ('Sam', 'ham', 'green'): 0.08333333333333333, ('Sam', 'ham', 'eggs'): 0.08333333333333333, ('Sam', 'ham', 'and'): 0.08333333333333333, ('Sam', 'ham', 'ham'): 0.08333333333333333, ('</s>', '<s>', '<s>'): 0.08333333333333333, ('</s>', '<s>', 'I'): 0.08333333333333333, ('</s>', '<s>', 'am'): 0.08333333333333333, ('</s>', '<s>', 'Sam'): 0.08333333333333333, ('</s>', '<s>', '</s>'): 0.08333333333333333, ('</s>', '<s>', 'do'): 0.08333333333333333, ('</s>', '<s>', 'not'): 0.08333333333333333, ('</s>', '<s>', 'like'): 0.08333333333333333, ('</s>', '<s>', 'green'): 0.08333333333333333, ('</s>', '<s>', 'eggs'): 0.08333333333333333, ('</s>', '<s>', 'and'): 0.08333333333333333, ('</s>', '<s>', 'ham'): 0.08333333333333333, ('</s>', 'I', '<s>'): 0.08333333333333333, ('</s>', 'I', 'I'): 0.08333333333333333, ('</s>', 'I', 'am'): 0.08333333333333333, ('</s>', 'I', 'Sam'): 0.08333333333333333, ('</s>', 'I', '</s>'): 0.08333333333333333, ('</s>', 'I', 'do'): 0.08333333333333333, ('</s>', 'I', 'not'): 0.08333333333333333, ('</s>', 'I', 'like'): 0.08333333333333333, ('</s>', 'I', 'green'): 0.08333333333333333, ('</s>', 'I', 'eggs'): 0.08333333333333333, ('</s>', 'I', 'and'): 0.08333333333333333, ('</s>', 'I', 'ham'): 0.08333333333333333, ('</s>', 'am', '<s>'): 0.08333333333333333, ('</s>', 'am', 'I'): 0.08333333333333333, ('</s>', 'am', 'am'): 0.08333333333333333, ('</s>', 'am', 'Sam'): 0.08333333333333333, ('</s>', 'am', '</s>'): 0.08333333333333333, ('</s>', 'am', 'do'): 0.08333333333333333, ('</s>', 'am', 'not'): 0.08333333333333333, ('</s>', 'am', 'like'): 0.08333333333333333, ('</s>', 'am', 'green'): 0.08333333333333333, ('</s>', 'am', 'eggs'): 0.08333333333333333, ('</s>', 'am', 'and'): 0.08333333333333333, ('</s>', 'am', 'ham'): 0.08333333333333333, ('</s>', 'Sam', '<s>'): 0.08333333333333333, ('</s>', 'Sam', 'I'): 0.08333333333333333, ('</s>', 'Sam', 'am'): 0.08333333333333333, ('</s>', 'Sam', 'Sam'): 0.08333333333333333, ('</s>', 'Sam', '</s>'): 0.08333333333333333, ('</s>', 'Sam', 'do'): 0.08333333333333333, ('</s>', 'Sam', 'not'): 0.08333333333333333, ('</s>', 'Sam', 'like'): 0.08333333333333333, ('</s>', 'Sam', 'green'): 0.08333333333333333, ('</s>', 'Sam', 'eggs'): 0.08333333333333333, ('</s>', 'Sam', 'and'): 0.08333333333333333, ('</s>', 'Sam', 'ham'): 0.08333333333333333, ('</s>', '</s>', '<s>'): 0.08333333333333333, ('</s>', '</s>', 'I'): 0.08333333333333333, ('</s>', '</s>', 'am'): 0.08333333333333333, ('</s>', '</s>', 'Sam'): 0.08333333333333333, ('</s>', '</s>', '</s>'): 0.08333333333333333, ('</s>', '</s>', 'do'): 0.08333333333333333, ('</s>', '</s>', 'not'): 0.08333333333333333, ('</s>', '</s>', 'like'): 0.08333333333333333, ('</s>', '</s>', 'green'): 0.08333333333333333, ('</s>', '</s>', 'eggs'): 0.08333333333333333, ('</s>', '</s>', 'and'): 0.08333333333333333, ('</s>', '</s>', 'ham'): 0.08333333333333333, ('</s>', 'do', '<s>'): 0.08333333333333333, ('</s>', 'do', 'I'): 0.08333333333333333, ('</s>', 'do', 'am'): 0.08333333333333333, ('</s>', 'do', 'Sam'): 0.08333333333333333, ('</s>', 'do', '</s>'): 0.08333333333333333, ('</s>', 'do', 'do'): 0.08333333333333333, ('</s>', 'do', 'not'): 0.08333333333333333, ('</s>', 'do', 'like'): 0.08333333333333333, ('</s>', 'do', 'green'): 0.08333333333333333, ('</s>', 'do', 'eggs'): 0.08333333333333333, ('</s>', 'do', 'and'): 0.08333333333333333, ('</s>', 'do', 'ham'): 0.08333333333333333, ('</s>', 'not', '<s>'): 0.08333333333333333, ('</s>', 'not', 'I'): 0.08333333333333333, ('</s>', 'not', 'am'): 0.08333333333333333, ('</s>', 'not', 'Sam'): 0.08333333333333333, ('</s>', 'not', '</s>'): 0.08333333333333333, ('</s>', 'not', 'do'): 0.08333333333333333, ('</s>', 'not', 'not'): 0.08333333333333333, ('</s>', 'not', 'like'): 0.08333333333333333, ('</s>', 'not', 'green'): 0.08333333333333333, ('</s>', 'not', 'eggs'): 0.08333333333333333, ('</s>', 'not', 'and'): 0.08333333333333333, ('</s>', 'not', 'ham'): 0.08333333333333333, ('</s>', 'like', '<s>'): 0.08333333333333333, ('</s>', 'like', 'I'): 0.08333333333333333, ('</s>', 'like', 'am'): 0.08333333333333333, ('</s>', 'like', 'Sam'): 0.08333333333333333, ('</s>', 'like', '</s>'): 0.08333333333333333, ('</s>', 'like', 'do'): 0.08333333333333333, ('</s>', 'like', 'not'): 0.08333333333333333, ('</s>', 'like', 'like'): 0.08333333333333333, ('</s>', 'like', 'green'): 0.08333333333333333, ('</s>', 'like', 'eggs'): 0.08333333333333333, ('</s>', 'like', 'and'): 0.08333333333333333, ('</s>', 'like', 'ham'): 0.08333333333333333, ('</s>', 'green', '<s>'): 0.08333333333333333, ('</s>', 'green', 'I'): 0.08333333333333333, ('</s>', 'green', 'am'): 0.08333333333333333, ('</s>', 'green', 'Sam'): 0.08333333333333333, ('</s>', 'green', '</s>'): 0.08333333333333333, ('</s>', 'green', 'do'): 0.08333333333333333, ('</s>', 'green', 'not'): 0.08333333333333333, ('</s>', 'green', 'like'): 0.08333333333333333, ('</s>', 'green', 'green'): 0.08333333333333333, ('</s>', 'green', 'eggs'): 0.08333333333333333, ('</s>', 'green', 'and'): 0.08333333333333333, ('</s>', 'green', 'ham'): 0.08333333333333333, ('</s>', 'eggs', '<s>'): 0.08333333333333333, ('</s>', 'eggs', 'I'): 0.08333333333333333, ('</s>', 'eggs', 'am'): 0.08333333333333333, ('</s>', 'eggs', 'Sam'): 0.08333333333333333, ('</s>', 'eggs', '</s>'): 0.08333333333333333, ('</s>', 'eggs', 'do'): 0.08333333333333333, ('</s>', 'eggs', 'not'): 0.08333333333333333, ('</s>', 'eggs', 'like'): 0.08333333333333333, ('</s>', 'eggs', 'green'): 0.08333333333333333, ('</s>', 'eggs', 'eggs'): 0.08333333333333333, ('</s>', 'eggs', 'and'): 0.08333333333333333, ('</s>', 'eggs', 'ham'): 0.08333333333333333, ('</s>', 'and', '<s>'): 0.08333333333333333, ('</s>', 'and', 'I'): 0.08333333333333333, ('</s>', 'and', 'am'): 0.08333333333333333, ('</s>', 'and', 'Sam'): 0.08333333333333333, ('</s>', 'and', '</s>'): 0.08333333333333333, ('</s>', 'and', 'do'): 0.08333333333333333, ('</s>', 'and', 'not'): 0.08333333333333333, ('</s>', 'and', 'like'): 0.08333333333333333, ('</s>', 'and', 'green'): 0.08333333333333333, ('</s>', 'and', 'eggs'): 0.08333333333333333, ('</s>', 'and', 'and'): 0.08333333333333333, ('</s>', 'and', 'ham'): 0.08333333333333333, ('</s>', 'ham', '<s>'): 0.08333333333333333, ('</s>', 'ham', 'I'): 0.08333333333333333, ('</s>', 'ham', 'am'): 0.08333333333333333, ('</s>', 'ham', 'Sam'): 0.08333333333333333, ('</s>', 'ham', '</s>'): 0.08333333333333333, ('</s>', 'ham', 'do'): 0.08333333333333333, ('</s>', 'ham', 'not'): 0.08333333333333333, ('</s>', 'ham', 'like'): 0.08333333333333333, ('</s>', 'ham', 'green'): 0.08333333333333333, ('</s>', 'ham', 'eggs'): 0.08333333333333333, ('</s>', 'ham', 'and'): 0.08333333333333333, ('</s>', 'ham', 'ham'): 0.08333333333333333, ('do', '<s>', '<s>'): 0.08333333333333333, ('do', '<s>', 'I'): 0.08333333333333333, ('do', '<s>', 'am'): 0.08333333333333333, ('do', '<s>', 'Sam'): 0.08333333333333333, ('do', '<s>', '</s>'): 0.08333333333333333, ('do', '<s>', 'do'): 0.08333333333333333, ('do', '<s>', 'not'): 0.08333333333333333, ('do', '<s>', 'like'): 0.08333333333333333, ('do', '<s>', 'green'): 0.08333333333333333, ('do', '<s>', 'eggs'): 0.08333333333333333, ('do', '<s>', 'and'): 0.08333333333333333, ('do', '<s>', 'ham'): 0.08333333333333333, ('do', 'I', '<s>'): 0.08333333333333333, ('do', 'I', 'I'): 0.08333333333333333, ('do', 'I', 'am'): 0.08333333333333333, ('do', 'I', 'Sam'): 0.08333333333333333, ('do', 'I', '</s>'): 0.08333333333333333, ('do', 'I', 'do'): 0.08333333333333333, ('do', 'I', 'not'): 0.08333333333333333, ('do', 'I', 'like'): 0.08333333333333333, ('do', 'I', 'green'): 0.08333333333333333, ('do', 'I', 'eggs'): 0.08333333333333333, ('do', 'I', 'and'): 0.08333333333333333, ('do', 'I', 'ham'): 0.08333333333333333, ('do', 'am', '<s>'): 0.08333333333333333, ('do', 'am', 'I'): 0.08333333333333333, ('do', 'am', 'am'): 0.08333333333333333, ('do', 'am', 'Sam'): 0.08333333333333333, ('do', 'am', '</s>'): 0.08333333333333333, ('do', 'am', 'do'): 0.08333333333333333, ('do', 'am', 'not'): 0.08333333333333333, ('do', 'am', 'like'): 0.08333333333333333, ('do', 'am', 'green'): 0.08333333333333333, ('do', 'am', 'eggs'): 0.08333333333333333, ('do', 'am', 'and'): 0.08333333333333333, ('do', 'am', 'ham'): 0.08333333333333333, ('do', 'Sam', '<s>'): 0.08333333333333333, ('do', 'Sam', 'I'): 0.08333333333333333, ('do', 'Sam', 'am'): 0.08333333333333333, ('do', 'Sam', 'Sam'): 0.08333333333333333, ('do', 'Sam', '</s>'): 0.08333333333333333, ('do', 'Sam', 'do'): 0.08333333333333333, ('do', 'Sam', 'not'): 0.08333333333333333, ('do', 'Sam', 'like'): 0.08333333333333333, ('do', 'Sam', 'green'): 0.08333333333333333, ('do', 'Sam', 'eggs'): 0.08333333333333333, ('do', 'Sam', 'and'): 0.08333333333333333, ('do', 'Sam', 'ham'): 0.08333333333333333, ('do', '</s>', '<s>'): 0.08333333333333333, ('do', '</s>', 'I'): 0.08333333333333333, ('do', '</s>', 'am'): 0.08333333333333333, ('do', '</s>', 'Sam'): 0.08333333333333333, ('do', '</s>', '</s>'): 0.08333333333333333, ('do', '</s>', 'do'): 0.08333333333333333, ('do', '</s>', 'not'): 0.08333333333333333, ('do', '</s>', 'like'): 0.08333333333333333, ('do', '</s>', 'green'): 0.08333333333333333, ('do', '</s>', 'eggs'): 0.08333333333333333, ('do', '</s>', 'and'): 0.08333333333333333, ('do', '</s>', 'ham'): 0.08333333333333333, ('do', 'do', '<s>'): 0.08333333333333333, ('do', 'do', 'I'): 0.08333333333333333, ('do', 'do', 'am'): 0.08333333333333333, ('do', 'do', 'Sam'): 0.08333333333333333, ('do', 'do', '</s>'): 0.08333333333333333, ('do', 'do', 'do'): 0.08333333333333333, ('do', 'do', 'not'): 0.08333333333333333, ('do', 'do', 'like'): 0.08333333333333333, ('do', 'do', 'green'): 0.08333333333333333, ('do', 'do', 'eggs'): 0.08333333333333333, ('do', 'do', 'and'): 0.08333333333333333, ('do', 'do', 'ham'): 0.08333333333333333, ('do', 'not', '<s>'): 0.07692307692307693, ('do', 'not', 'I'): 0.07692307692307693, ('do', 'not', 'am'): 0.07692307692307693, ('do', 'not', 'Sam'): 0.07692307692307693, ('do', 'not', '</s>'): 0.07692307692307693, ('do', 'not', 'do'): 0.07692307692307693, ('do', 'not', 'not'): 0.07692307692307693, ('do', 'not', 'green'): 0.07692307692307693, ('do', 'not', 'eggs'): 0.07692307692307693, ('do', 'not', 'and'): 0.07692307692307693, ('do', 'not', 'ham'): 0.07692307692307693, ('do', 'like', '<s>'): 0.08333333333333333, ('do', 'like', 'I'): 0.08333333333333333, ('do', 'like', 'am'): 0.08333333333333333, ('do', 'like', 'Sam'): 0.08333333333333333, ('do', 'like', '</s>'): 0.08333333333333333, ('do', 'like', 'do'): 0.08333333333333333, ('do', 'like', 'not'): 0.08333333333333333, ('do', 'like', 'like'): 0.08333333333333333, ('do', 'like', 'green'): 0.08333333333333333, ('do', 'like', 'eggs'): 0.08333333333333333, ('do', 'like', 'and'): 0.08333333333333333, ('do', 'like', 'ham'): 0.08333333333333333, ('do', 'green', '<s>'): 0.08333333333333333, ('do', 'green', 'I'): 0.08333333333333333, ('do', 'green', 'am'): 0.08333333333333333, ('do', 'green', 'Sam'): 0.08333333333333333, ('do', 'green', '</s>'): 0.08333333333333333, ('do', 'green', 'do'): 0.08333333333333333, ('do', 'green', 'not'): 0.08333333333333333, ('do', 'green', 'like'): 0.08333333333333333, ('do', 'green', 'green'): 0.08333333333333333, ('do', 'green', 'eggs'): 0.08333333333333333, ('do', 'green', 'and'): 0.08333333333333333, ('do', 'green', 'ham'): 0.08333333333333333, ('do', 'eggs', '<s>'): 0.08333333333333333, ('do', 'eggs', 'I'): 0.08333333333333333, ('do', 'eggs', 'am'): 0.08333333333333333, ('do', 'eggs', 'Sam'): 0.08333333333333333, ('do', 'eggs', '</s>'): 0.08333333333333333, ('do', 'eggs', 'do'): 0.08333333333333333, ('do', 'eggs', 'not'): 0.08333333333333333, ('do', 'eggs', 'like'): 0.08333333333333333, ('do', 'eggs', 'green'): 0.08333333333333333, ('do', 'eggs', 'eggs'): 0.08333333333333333, ('do', 'eggs', 'and'): 0.08333333333333333, ('do', 'eggs', 'ham'): 0.08333333333333333, ('do', 'and', '<s>'): 0.08333333333333333, ('do', 'and', 'I'): 0.08333333333333333, ('do', 'and', 'am'): 0.08333333333333333, ('do', 'and', 'Sam'): 0.08333333333333333, ('do', 'and', '</s>'): 0.08333333333333333, ('do', 'and', 'do'): 0.08333333333333333, ('do', 'and', 'not'): 0.08333333333333333, ('do', 'and', 'like'): 0.08333333333333333, ('do', 'and', 'green'): 0.08333333333333333, ('do', 'and', 'eggs'): 0.08333333333333333, ('do', 'and', 'and'): 0.08333333333333333, ('do', 'and', 'ham'): 0.08333333333333333, ('do', 'ham', '<s>'): 0.08333333333333333, ('do', 'ham', 'I'): 0.08333333333333333, ('do', 'ham', 'am'): 0.08333333333333333, ('do', 'ham', 'Sam'): 0.08333333333333333, ('do', 'ham', '</s>'): 0.08333333333333333, ('do', 'ham', 'do'): 0.08333333333333333, ('do', 'ham', 'not'): 0.08333333333333333, ('do', 'ham', 'like'): 0.08333333333333333, ('do', 'ham', 'green'): 0.08333333333333333, ('do', 'ham', 'eggs'): 0.08333333333333333, ('do', 'ham', 'and'): 0.08333333333333333, ('do', 'ham', 'ham'): 0.08333333333333333, ('not', '<s>', '<s>'): 0.08333333333333333, ('not', '<s>', 'I'): 0.08333333333333333, ('not', '<s>', 'am'): 0.08333333333333333, ('not', '<s>', 'Sam'): 0.08333333333333333, ('not', '<s>', '</s>'): 0.08333333333333333, ('not', '<s>', 'do'): 0.08333333333333333, ('not', '<s>', 'not'): 0.08333333333333333, ('not', '<s>', 'like'): 0.08333333333333333, ('not', '<s>', 'green'): 0.08333333333333333, ('not', '<s>', 'eggs'): 0.08333333333333333, ('not', '<s>', 'and'): 0.08333333333333333, ('not', '<s>', 'ham'): 0.08333333333333333, ('not', 'I', '<s>'): 0.08333333333333333, ('not', 'I', 'I'): 0.08333333333333333, ('not', 'I', 'am'): 0.08333333333333333, ('not', 'I', 'Sam'): 0.08333333333333333, ('not', 'I', '</s>'): 0.08333333333333333, ('not', 'I', 'do'): 0.08333333333333333, ('not', 'I', 'not'): 0.08333333333333333, ('not', 'I', 'like'): 0.08333333333333333, ('not', 'I', 'green'): 0.08333333333333333, ('not', 'I', 'eggs'): 0.08333333333333333, ('not', 'I', 'and'): 0.08333333333333333, ('not', 'I', 'ham'): 0.08333333333333333, ('not', 'am', '<s>'): 0.08333333333333333, ('not', 'am', 'I'): 0.08333333333333333, ('not', 'am', 'am'): 0.08333333333333333, ('not', 'am', 'Sam'): 0.08333333333333333, ('not', 'am', '</s>'): 0.08333333333333333, ('not', 'am', 'do'): 0.08333333333333333, ('not', 'am', 'not'): 0.08333333333333333, ('not', 'am', 'like'): 0.08333333333333333, ('not', 'am', 'green'): 0.08333333333333333, ('not', 'am', 'eggs'): 0.08333333333333333, ('not', 'am', 'and'): 0.08333333333333333, ('not', 'am', 'ham'): 0.08333333333333333, ('not', 'Sam', '<s>'): 0.08333333333333333, ('not', 'Sam', 'I'): 0.08333333333333333, ('not', 'Sam', 'am'): 0.08333333333333333, ('not', 'Sam', 'Sam'): 0.08333333333333333, ('not', 'Sam', '</s>'): 0.08333333333333333, ('not', 'Sam', 'do'): 0.08333333333333333, ('not', 'Sam', 'not'): 0.08333333333333333, ('not', 'Sam', 'like'): 0.08333333333333333, ('not', 'Sam', 'green'): 0.08333333333333333, ('not', 'Sam', 'eggs'): 0.08333333333333333, ('not', 'Sam', 'and'): 0.08333333333333333, ('not', 'Sam', 'ham'): 0.08333333333333333, ('not', '</s>', '<s>'): 0.08333333333333333, ('not', '</s>', 'I'): 0.08333333333333333, ('not', '</s>', 'am'): 0.08333333333333333, ('not', '</s>', 'Sam'): 0.08333333333333333, ('not', '</s>', '</s>'): 0.08333333333333333, ('not', '</s>', 'do'): 0.08333333333333333, ('not', '</s>', 'not'): 0.08333333333333333, ('not', '</s>', 'like'): 0.08333333333333333, ('not', '</s>', 'green'): 0.08333333333333333, ('not', '</s>', 'eggs'): 0.08333333333333333, ('not', '</s>', 'and'): 0.08333333333333333, ('not', '</s>', 'ham'): 0.08333333333333333, ('not', 'do', '<s>'): 0.08333333333333333, ('not', 'do', 'I'): 0.08333333333333333, ('not', 'do', 'am'): 0.08333333333333333, ('not', 'do', 'Sam'): 0.08333333333333333, ('not', 'do', '</s>'): 0.08333333333333333, ('not', 'do', 'do'): 0.08333333333333333, ('not', 'do', 'not'): 0.08333333333333333, ('not', 'do', 'like'): 0.08333333333333333, ('not', 'do', 'green'): 0.08333333333333333, ('not', 'do', 'eggs'): 0.08333333333333333, ('not', 'do', 'and'): 0.08333333333333333, ('not', 'do', 'ham'): 0.08333333333333333, ('not', 'not', '<s>'): 0.08333333333333333, ('not', 'not', 'I'): 0.08333333333333333, ('not', 'not', 'am'): 0.08333333333333333, ('not', 'not', 'Sam'): 0.08333333333333333, ('not', 'not', '</s>'): 0.08333333333333333, ('not', 'not', 'do'): 0.08333333333333333, ('not', 'not', 'not'): 0.08333333333333333, ('not', 'not', 'like'): 0.08333333333333333, ('not', 'not', 'green'): 0.08333333333333333, ('not', 'not', 'eggs'): 0.08333333333333333, ('not', 'not', 'and'): 0.08333333333333333, ('not', 'not', 'ham'): 0.08333333333333333, ('not', 'like', '<s>'): 0.07692307692307693, ('not', 'like', 'I'): 0.07692307692307693, ('not', 'like', 'am'): 0.07692307692307693, ('not', 'like', 'Sam'): 0.07692307692307693, ('not', 'like', '</s>'): 0.07692307692307693, ('not', 'like', 'do'): 0.07692307692307693, ('not', 'like', 'not'): 0.07692307692307693, ('not', 'like', 'like'): 0.07692307692307693, ('not', 'like', 'eggs'): 0.07692307692307693, ('not', 'like', 'and'): 0.07692307692307693, ('not', 'like', 'ham'): 0.07692307692307693, ('not', 'green', '<s>'): 0.08333333333333333, ('not', 'green', 'I'): 0.08333333333333333, ('not', 'green', 'am'): 0.08333333333333333, ('not', 'green', 'Sam'): 0.08333333333333333, ('not', 'green', '</s>'): 0.08333333333333333, ('not', 'green', 'do'): 0.08333333333333333, ('not', 'green', 'not'): 0.08333333333333333, ('not', 'green', 'like'): 0.08333333333333333, ('not', 'green', 'green'): 0.08333333333333333, ('not', 'green', 'eggs'): 0.08333333333333333, ('not', 'green', 'and'): 0.08333333333333333, ('not', 'green', 'ham'): 0.08333333333333333, ('not', 'eggs', '<s>'): 0.08333333333333333, ('not', 'eggs', 'I'): 0.08333333333333333, ('not', 'eggs', 'am'): 0.08333333333333333, ('not', 'eggs', 'Sam'): 0.08333333333333333, ('not', 'eggs', '</s>'): 0.08333333333333333, ('not', 'eggs', 'do'): 0.08333333333333333, ('not', 'eggs', 'not'): 0.08333333333333333, ('not', 'eggs', 'like'): 0.08333333333333333, ('not', 'eggs', 'green'): 0.08333333333333333, ('not', 'eggs', 'eggs'): 0.08333333333333333, ('not', 'eggs', 'and'): 0.08333333333333333, ('not', 'eggs', 'ham'): 0.08333333333333333, ('not', 'and', '<s>'): 0.08333333333333333, ('not', 'and', 'I'): 0.08333333333333333, ('not', 'and', 'am'): 0.08333333333333333, ('not', 'and', 'Sam'): 0.08333333333333333, ('not', 'and', '</s>'): 0.08333333333333333, ('not', 'and', 'do'): 0.08333333333333333, ('not', 'and', 'not'): 0.08333333333333333, ('not', 'and', 'like'): 0.08333333333333333, ('not', 'and', 'green'): 0.08333333333333333, ('not', 'and', 'eggs'): 0.08333333333333333, ('not', 'and', 'and'): 0.08333333333333333, ('not', 'and', 'ham'): 0.08333333333333333, ('not', 'ham', '<s>'): 0.08333333333333333, ('not', 'ham', 'I'): 0.08333333333333333, ('not', 'ham', 'am'): 0.08333333333333333, ('not', 'ham', 'Sam'): 0.08333333333333333, ('not', 'ham', '</s>'): 0.08333333333333333, ('not', 'ham', 'do'): 0.08333333333333333, ('not', 'ham', 'not'): 0.08333333333333333, ('not', 'ham', 'like'): 0.08333333333333333, ('not', 'ham', 'green'): 0.08333333333333333, ('not', 'ham', 'eggs'): 0.08333333333333333, ('not', 'ham', 'and'): 0.08333333333333333, ('not', 'ham', 'ham'): 0.08333333333333333, ('like', '<s>', '<s>'): 0.08333333333333333, ('like', '<s>', 'I'): 0.08333333333333333, ('like', '<s>', 'am'): 0.08333333333333333, ('like', '<s>', 'Sam'): 0.08333333333333333, ('like', '<s>', '</s>'): 0.08333333333333333, ('like', '<s>', 'do'): 0.08333333333333333, ('like', '<s>', 'not'): 0.08333333333333333, ('like', '<s>', 'like'): 0.08333333333333333, ('like', '<s>', 'green'): 0.08333333333333333, ('like', '<s>', 'eggs'): 0.08333333333333333, ('like', '<s>', 'and'): 0.08333333333333333, ('like', '<s>', 'ham'): 0.08333333333333333, ('like', 'I', '<s>'): 0.08333333333333333, ('like', 'I', 'I'): 0.08333333333333333, ('like', 'I', 'am'): 0.08333333333333333, ('like', 'I', 'Sam'): 0.08333333333333333, ('like', 'I', '</s>'): 0.08333333333333333, ('like', 'I', 'do'): 0.08333333333333333, ('like', 'I', 'not'): 0.08333333333333333, ('like', 'I', 'like'): 0.08333333333333333, ('like', 'I', 'green'): 0.08333333333333333, ('like', 'I', 'eggs'): 0.08333333333333333, ('like', 'I', 'and'): 0.08333333333333333, ('like', 'I', 'ham'): 0.08333333333333333, ('like', 'am', '<s>'): 0.08333333333333333, ('like', 'am', 'I'): 0.08333333333333333, ('like', 'am', 'am'): 0.08333333333333333, ('like', 'am', 'Sam'): 0.08333333333333333, ('like', 'am', '</s>'): 0.08333333333333333, ('like', 'am', 'do'): 0.08333333333333333, ('like', 'am', 'not'): 0.08333333333333333, ('like', 'am', 'like'): 0.08333333333333333, ('like', 'am', 'green'): 0.08333333333333333, ('like', 'am', 'eggs'): 0.08333333333333333, ('like', 'am', 'and'): 0.08333333333333333, ('like', 'am', 'ham'): 0.08333333333333333, ('like', 'Sam', '<s>'): 0.08333333333333333, ('like', 'Sam', 'I'): 0.08333333333333333, ('like', 'Sam', 'am'): 0.08333333333333333, ('like', 'Sam', 'Sam'): 0.08333333333333333, ('like', 'Sam', '</s>'): 0.08333333333333333, ('like', 'Sam', 'do'): 0.08333333333333333, ('like', 'Sam', 'not'): 0.08333333333333333, ('like', 'Sam', 'like'): 0.08333333333333333, ('like', 'Sam', 'green'): 0.08333333333333333, ('like', 'Sam', 'eggs'): 0.08333333333333333, ('like', 'Sam', 'and'): 0.08333333333333333, ('like', 'Sam', 'ham'): 0.08333333333333333, ('like', '</s>', '<s>'): 0.08333333333333333, ('like', '</s>', 'I'): 0.08333333333333333, ('like', '</s>', 'am'): 0.08333333333333333, ('like', '</s>', 'Sam'): 0.08333333333333333, ('like', '</s>', '</s>'): 0.08333333333333333, ('like', '</s>', 'do'): 0.08333333333333333, ('like', '</s>', 'not'): 0.08333333333333333, ('like', '</s>', 'like'): 0.08333333333333333, ('like', '</s>', 'green'): 0.08333333333333333, ('like', '</s>', 'eggs'): 0.08333333333333333, ('like', '</s>', 'and'): 0.08333333333333333, ('like', '</s>', 'ham'): 0.08333333333333333, ('like', 'do', '<s>'): 0.08333333333333333, ('like', 'do', 'I'): 0.08333333333333333, ('like', 'do', 'am'): 0.08333333333333333, ('like', 'do', 'Sam'): 0.08333333333333333, ('like', 'do', '</s>'): 0.08333333333333333, ('like', 'do', 'do'): 0.08333333333333333, ('like', 'do', 'not'): 0.08333333333333333, ('like', 'do', 'like'): 0.08333333333333333, ('like', 'do', 'green'): 0.08333333333333333, ('like', 'do', 'eggs'): 0.08333333333333333, ('like', 'do', 'and'): 0.08333333333333333, ('like', 'do', 'ham'): 0.08333333333333333, ('like', 'not', '<s>'): 0.08333333333333333, ('like', 'not', 'I'): 0.08333333333333333, ('like', 'not', 'am'): 0.08333333333333333, ('like', 'not', 'Sam'): 0.08333333333333333, ('like', 'not', '</s>'): 0.08333333333333333, ('like', 'not', 'do'): 0.08333333333333333, ('like', 'not', 'not'): 0.08333333333333333, ('like', 'not', 'like'): 0.08333333333333333, ('like', 'not', 'green'): 0.08333333333333333, ('like', 'not', 'eggs'): 0.08333333333333333, ('like', 'not', 'and'): 0.08333333333333333, ('like', 'not', 'ham'): 0.08333333333333333, ('like', 'like', '<s>'): 0.08333333333333333, ('like', 'like', 'I'): 0.08333333333333333, ('like', 'like', 'am'): 0.08333333333333333, ('like', 'like', 'Sam'): 0.08333333333333333, ('like', 'like', '</s>'): 0.08333333333333333, ('like', 'like', 'do'): 0.08333333333333333, ('like', 'like', 'not'): 0.08333333333333333, ('like', 'like', 'like'): 0.08333333333333333, ('like', 'like', 'green'): 0.08333333333333333, ('like', 'like', 'eggs'): 0.08333333333333333, ('like', 'like', 'and'): 0.08333333333333333, ('like', 'like', 'ham'): 0.08333333333333333, ('like', 'green', '<s>'): 0.07692307692307693, ('like', 'green', 'I'): 0.07692307692307693, ('like', 'green', 'am'): 0.07692307692307693, ('like', 'green', 'Sam'): 0.07692307692307693, ('like', 'green', '</s>'): 0.07692307692307693, ('like', 'green', 'do'): 0.07692307692307693, ('like', 'green', 'not'): 0.07692307692307693, ('like', 'green', 'like'): 0.07692307692307693, ('like', 'green', 'green'): 0.07692307692307693, ('like', 'green', 'and'): 0.07692307692307693, ('like', 'green', 'ham'): 0.07692307692307693, ('like', 'eggs', '<s>'): 0.08333333333333333, ('like', 'eggs', 'I'): 0.08333333333333333, ('like', 'eggs', 'am'): 0.08333333333333333, ('like', 'eggs', 'Sam'): 0.08333333333333333, ('like', 'eggs', '</s>'): 0.08333333333333333, ('like', 'eggs', 'do'): 0.08333333333333333, ('like', 'eggs', 'not'): 0.08333333333333333, ('like', 'eggs', 'like'): 0.08333333333333333, ('like', 'eggs', 'green'): 0.08333333333333333, ('like', 'eggs', 'eggs'): 0.08333333333333333, ('like', 'eggs', 'and'): 0.08333333333333333, ('like', 'eggs', 'ham'): 0.08333333333333333, ('like', 'and', '<s>'): 0.08333333333333333, ('like', 'and', 'I'): 0.08333333333333333, ('like', 'and', 'am'): 0.08333333333333333, ('like', 'and', 'Sam'): 0.08333333333333333, ('like', 'and', '</s>'): 0.08333333333333333, ('like', 'and', 'do'): 0.08333333333333333, ('like', 'and', 'not'): 0.08333333333333333, ('like', 'and', 'like'): 0.08333333333333333, ('like', 'and', 'green'): 0.08333333333333333, ('like', 'and', 'eggs'): 0.08333333333333333, ('like', 'and', 'and'): 0.08333333333333333, ('like', 'and', 'ham'): 0.08333333333333333, ('like', 'ham', '<s>'): 0.08333333333333333, ('like', 'ham', 'I'): 0.08333333333333333, ('like', 'ham', 'am'): 0.08333333333333333, ('like', 'ham', 'Sam'): 0.08333333333333333, ('like', 'ham', '</s>'): 0.08333333333333333, ('like', 'ham', 'do'): 0.08333333333333333, ('like', 'ham', 'not'): 0.08333333333333333, ('like', 'ham', 'like'): 0.08333333333333333, ('like', 'ham', 'green'): 0.08333333333333333, ('like', 'ham', 'eggs'): 0.08333333333333333, ('like', 'ham', 'and'): 0.08333333333333333, ('like', 'ham', 'ham'): 0.08333333333333333, ('green', '<s>', '<s>'): 0.08333333333333333, ('green', '<s>', 'I'): 0.08333333333333333, ('green', '<s>', 'am'): 0.08333333333333333, ('green', '<s>', 'Sam'): 0.08333333333333333, ('green', '<s>', '</s>'): 0.08333333333333333, ('green', '<s>', 'do'): 0.08333333333333333, ('green', '<s>', 'not'): 0.08333333333333333, ('green', '<s>', 'like'): 0.08333333333333333, ('green', '<s>', 'green'): 0.08333333333333333, ('green', '<s>', 'eggs'): 0.08333333333333333, ('green', '<s>', 'and'): 0.08333333333333333, ('green', '<s>', 'ham'): 0.08333333333333333, ('green', 'I', '<s>'): 0.08333333333333333, ('green', 'I', 'I'): 0.08333333333333333, ('green', 'I', 'am'): 0.08333333333333333, ('green', 'I', 'Sam'): 0.08333333333333333, ('green', 'I', '</s>'): 0.08333333333333333, ('green', 'I', 'do'): 0.08333333333333333, ('green', 'I', 'not'): 0.08333333333333333, ('green', 'I', 'like'): 0.08333333333333333, ('green', 'I', 'green'): 0.08333333333333333, ('green', 'I', 'eggs'): 0.08333333333333333, ('green', 'I', 'and'): 0.08333333333333333, ('green', 'I', 'ham'): 0.08333333333333333, ('green', 'am', '<s>'): 0.08333333333333333, ('green', 'am', 'I'): 0.08333333333333333, ('green', 'am', 'am'): 0.08333333333333333, ('green', 'am', 'Sam'): 0.08333333333333333, ('green', 'am', '</s>'): 0.08333333333333333, ('green', 'am', 'do'): 0.08333333333333333, ('green', 'am', 'not'): 0.08333333333333333, ('green', 'am', 'like'): 0.08333333333333333, ('green', 'am', 'green'): 0.08333333333333333, ('green', 'am', 'eggs'): 0.08333333333333333, ('green', 'am', 'and'): 0.08333333333333333, ('green', 'am', 'ham'): 0.08333333333333333, ('green', 'Sam', '<s>'): 0.08333333333333333, ('green', 'Sam', 'I'): 0.08333333333333333, ('green', 'Sam', 'am'): 0.08333333333333333, ('green', 'Sam', 'Sam'): 0.08333333333333333, ('green', 'Sam', '</s>'): 0.08333333333333333, ('green', 'Sam', 'do'): 0.08333333333333333, ('green', 'Sam', 'not'): 0.08333333333333333, ('green', 'Sam', 'like'): 0.08333333333333333, ('green', 'Sam', 'green'): 0.08333333333333333, ('green', 'Sam', 'eggs'): 0.08333333333333333, ('green', 'Sam', 'and'): 0.08333333333333333, ('green', 'Sam', 'ham'): 0.08333333333333333, ('green', '</s>', '<s>'): 0.08333333333333333, ('green', '</s>', 'I'): 0.08333333333333333, ('green', '</s>', 'am'): 0.08333333333333333, ('green', '</s>', 'Sam'): 0.08333333333333333, ('green', '</s>', '</s>'): 0.08333333333333333, ('green', '</s>', 'do'): 0.08333333333333333, ('green', '</s>', 'not'): 0.08333333333333333, ('green', '</s>', 'like'): 0.08333333333333333, ('green', '</s>', 'green'): 0.08333333333333333, ('green', '</s>', 'eggs'): 0.08333333333333333, ('green', '</s>', 'and'): 0.08333333333333333, ('green', '</s>', 'ham'): 0.08333333333333333, ('green', 'do', '<s>'): 0.08333333333333333, ('green', 'do', 'I'): 0.08333333333333333, ('green', 'do', 'am'): 0.08333333333333333, ('green', 'do', 'Sam'): 0.08333333333333333, ('green', 'do', '</s>'): 0.08333333333333333, ('green', 'do', 'do'): 0.08333333333333333, ('green', 'do', 'not'): 0.08333333333333333, ('green', 'do', 'like'): 0.08333333333333333, ('green', 'do', 'green'): 0.08333333333333333, ('green', 'do', 'eggs'): 0.08333333333333333, ('green', 'do', 'and'): 0.08333333333333333, ('green', 'do', 'ham'): 0.08333333333333333, ('green', 'not', '<s>'): 0.08333333333333333, ('green', 'not', 'I'): 0.08333333333333333, ('green', 'not', 'am'): 0.08333333333333333, ('green', 'not', 'Sam'): 0.08333333333333333, ('green', 'not', '</s>'): 0.08333333333333333, ('green', 'not', 'do'): 0.08333333333333333, ('green', 'not', 'not'): 0.08333333333333333, ('green', 'not', 'like'): 0.08333333333333333, ('green', 'not', 'green'): 0.08333333333333333, ('green', 'not', 'eggs'): 0.08333333333333333, ('green', 'not', 'and'): 0.08333333333333333, ('green', 'not', 'ham'): 0.08333333333333333, ('green', 'like', '<s>'): 0.08333333333333333, ('green', 'like', 'I'): 0.08333333333333333, ('green', 'like', 'am'): 0.08333333333333333, ('green', 'like', 'Sam'): 0.08333333333333333, ('green', 'like', '</s>'): 0.08333333333333333, ('green', 'like', 'do'): 0.08333333333333333, ('green', 'like', 'not'): 0.08333333333333333, ('green', 'like', 'like'): 0.08333333333333333, ('green', 'like', 'green'): 0.08333333333333333, ('green', 'like', 'eggs'): 0.08333333333333333, ('green', 'like', 'and'): 0.08333333333333333, ('green', 'like', 'ham'): 0.08333333333333333, ('green', 'green', '<s>'): 0.08333333333333333, ('green', 'green', 'I'): 0.08333333333333333, ('green', 'green', 'am'): 0.08333333333333333, ('green', 'green', 'Sam'): 0.08333333333333333, ('green', 'green', '</s>'): 0.08333333333333333, ('green', 'green', 'do'): 0.08333333333333333, ('green', 'green', 'not'): 0.08333333333333333, ('green', 'green', 'like'): 0.08333333333333333, ('green', 'green', 'green'): 0.08333333333333333, ('green', 'green', 'eggs'): 0.08333333333333333, ('green', 'green', 'and'): 0.08333333333333333, ('green', 'green', 'ham'): 0.08333333333333333, ('green', 'eggs', '<s>'): 0.07692307692307693, ('green', 'eggs', 'I'): 0.07692307692307693, ('green', 'eggs', 'am'): 0.07692307692307693, ('green', 'eggs', 'Sam'): 0.07692307692307693, ('green', 'eggs', '</s>'): 0.07692307692307693, ('green', 'eggs', 'do'): 0.07692307692307693, ('green', 'eggs', 'not'): 0.07692307692307693, ('green', 'eggs', 'like'): 0.07692307692307693, ('green', 'eggs', 'green'): 0.07692307692307693, ('green', 'eggs', 'eggs'): 0.07692307692307693, ('green', 'eggs', 'ham'): 0.07692307692307693, ('green', 'and', '<s>'): 0.08333333333333333, ('green', 'and', 'I'): 0.08333333333333333, ('green', 'and', 'am'): 0.08333333333333333, ('green', 'and', 'Sam'): 0.08333333333333333, ('green', 'and', '</s>'): 0.08333333333333333, ('green', 'and', 'do'): 0.08333333333333333, ('green', 'and', 'not'): 0.08333333333333333, ('green', 'and', 'like'): 0.08333333333333333, ('green', 'and', 'green'): 0.08333333333333333, ('green', 'and', 'eggs'): 0.08333333333333333, ('green', 'and', 'and'): 0.08333333333333333, ('green', 'and', 'ham'): 0.08333333333333333, ('green', 'ham', '<s>'): 0.08333333333333333, ('green', 'ham', 'I'): 0.08333333333333333, ('green', 'ham', 'am'): 0.08333333333333333, ('green', 'ham', 'Sam'): 0.08333333333333333, ('green', 'ham', '</s>'): 0.08333333333333333, ('green', 'ham', 'do'): 0.08333333333333333, ('green', 'ham', 'not'): 0.08333333333333333, ('green', 'ham', 'like'): 0.08333333333333333, ('green', 'ham', 'green'): 0.08333333333333333, ('green', 'ham', 'eggs'): 0.08333333333333333, ('green', 'ham', 'and'): 0.08333333333333333, ('green', 'ham', 'ham'): 0.08333333333333333, ('eggs', '<s>', '<s>'): 0.08333333333333333, ('eggs', '<s>', 'I'): 0.08333333333333333, ('eggs', '<s>', 'am'): 0.08333333333333333, ('eggs', '<s>', 'Sam'): 0.08333333333333333, ('eggs', '<s>', '</s>'): 0.08333333333333333, ('eggs', '<s>', 'do'): 0.08333333333333333, ('eggs', '<s>', 'not'): 0.08333333333333333, ('eggs', '<s>', 'like'): 0.08333333333333333, ('eggs', '<s>', 'green'): 0.08333333333333333, ('eggs', '<s>', 'eggs'): 0.08333333333333333, ('eggs', '<s>', 'and'): 0.08333333333333333, ('eggs', '<s>', 'ham'): 0.08333333333333333, ('eggs', 'I', '<s>'): 0.08333333333333333, ('eggs', 'I', 'I'): 0.08333333333333333, ('eggs', 'I', 'am'): 0.08333333333333333, ('eggs', 'I', 'Sam'): 0.08333333333333333, ('eggs', 'I', '</s>'): 0.08333333333333333, ('eggs', 'I', 'do'): 0.08333333333333333, ('eggs', 'I', 'not'): 0.08333333333333333, ('eggs', 'I', 'like'): 0.08333333333333333, ('eggs', 'I', 'green'): 0.08333333333333333, ('eggs', 'I', 'eggs'): 0.08333333333333333, ('eggs', 'I', 'and'): 0.08333333333333333, ('eggs', 'I', 'ham'): 0.08333333333333333, ('eggs', 'am', '<s>'): 0.08333333333333333, ('eggs', 'am', 'I'): 0.08333333333333333, ('eggs', 'am', 'am'): 0.08333333333333333, ('eggs', 'am', 'Sam'): 0.08333333333333333, ('eggs', 'am', '</s>'): 0.08333333333333333, ('eggs', 'am', 'do'): 0.08333333333333333, ('eggs', 'am', 'not'): 0.08333333333333333, ('eggs', 'am', 'like'): 0.08333333333333333, ('eggs', 'am', 'green'): 0.08333333333333333, ('eggs', 'am', 'eggs'): 0.08333333333333333, ('eggs', 'am', 'and'): 0.08333333333333333, ('eggs', 'am', 'ham'): 0.08333333333333333, ('eggs', 'Sam', '<s>'): 0.08333333333333333, ('eggs', 'Sam', 'I'): 0.08333333333333333, ('eggs', 'Sam', 'am'): 0.08333333333333333, ('eggs', 'Sam', 'Sam'): 0.08333333333333333, ('eggs', 'Sam', '</s>'): 0.08333333333333333, ('eggs', 'Sam', 'do'): 0.08333333333333333, ('eggs', 'Sam', 'not'): 0.08333333333333333, ('eggs', 'Sam', 'like'): 0.08333333333333333, ('eggs', 'Sam', 'green'): 0.08333333333333333, ('eggs', 'Sam', 'eggs'): 0.08333333333333333, ('eggs', 'Sam', 'and'): 0.08333333333333333, ('eggs', 'Sam', 'ham'): 0.08333333333333333, ('eggs', '</s>', '<s>'): 0.08333333333333333, ('eggs', '</s>', 'I'): 0.08333333333333333, ('eggs', '</s>', 'am'): 0.08333333333333333, ('eggs', '</s>', 'Sam'): 0.08333333333333333, ('eggs', '</s>', '</s>'): 0.08333333333333333, ('eggs', '</s>', 'do'): 0.08333333333333333, ('eggs', '</s>', 'not'): 0.08333333333333333, ('eggs', '</s>', 'like'): 0.08333333333333333, ('eggs', '</s>', 'green'): 0.08333333333333333, ('eggs', '</s>', 'eggs'): 0.08333333333333333, ('eggs', '</s>', 'and'): 0.08333333333333333, ('eggs', '</s>', 'ham'): 0.08333333333333333, ('eggs', 'do', '<s>'): 0.08333333333333333, ('eggs', 'do', 'I'): 0.08333333333333333, ('eggs', 'do', 'am'): 0.08333333333333333, ('eggs', 'do', 'Sam'): 0.08333333333333333, ('eggs', 'do', '</s>'): 0.08333333333333333, ('eggs', 'do', 'do'): 0.08333333333333333, ('eggs', 'do', 'not'): 0.08333333333333333, ('eggs', 'do', 'like'): 0.08333333333333333, ('eggs', 'do', 'green'): 0.08333333333333333, ('eggs', 'do', 'eggs'): 0.08333333333333333, ('eggs', 'do', 'and'): 0.08333333333333333, ('eggs', 'do', 'ham'): 0.08333333333333333, ('eggs', 'not', '<s>'): 0.08333333333333333, ('eggs', 'not', 'I'): 0.08333333333333333, ('eggs', 'not', 'am'): 0.08333333333333333, ('eggs', 'not', 'Sam'): 0.08333333333333333, ('eggs', 'not', '</s>'): 0.08333333333333333, ('eggs', 'not', 'do'): 0.08333333333333333, ('eggs', 'not', 'not'): 0.08333333333333333, ('eggs', 'not', 'like'): 0.08333333333333333, ('eggs', 'not', 'green'): 0.08333333333333333, ('eggs', 'not', 'eggs'): 0.08333333333333333, ('eggs', 'not', 'and'): 0.08333333333333333, ('eggs', 'not', 'ham'): 0.08333333333333333, ('eggs', 'like', '<s>'): 0.08333333333333333, ('eggs', 'like', 'I'): 0.08333333333333333, ('eggs', 'like', 'am'): 0.08333333333333333, ('eggs', 'like', 'Sam'): 0.08333333333333333, ('eggs', 'like', '</s>'): 0.08333333333333333, ('eggs', 'like', 'do'): 0.08333333333333333, ('eggs', 'like', 'not'): 0.08333333333333333, ('eggs', 'like', 'like'): 0.08333333333333333, ('eggs', 'like', 'green'): 0.08333333333333333, ('eggs', 'like', 'eggs'): 0.08333333333333333, ('eggs', 'like', 'and'): 0.08333333333333333, ('eggs', 'like', 'ham'): 0.08333333333333333, ('eggs', 'green', '<s>'): 0.08333333333333333, ('eggs', 'green', 'I'): 0.08333333333333333, ('eggs', 'green', 'am'): 0.08333333333333333, ('eggs', 'green', 'Sam'): 0.08333333333333333, ('eggs', 'green', '</s>'): 0.08333333333333333, ('eggs', 'green', 'do'): 0.08333333333333333, ('eggs', 'green', 'not'): 0.08333333333333333, ('eggs', 'green', 'like'): 0.08333333333333333, ('eggs', 'green', 'green'): 0.08333333333333333, ('eggs', 'green', 'eggs'): 0.08333333333333333, ('eggs', 'green', 'and'): 0.08333333333333333, ('eggs', 'green', 'ham'): 0.08333333333333333, ('eggs', 'eggs', '<s>'): 0.08333333333333333, ('eggs', 'eggs', 'I'): 0.08333333333333333, ('eggs', 'eggs', 'am'): 0.08333333333333333, ('eggs', 'eggs', 'Sam'): 0.08333333333333333, ('eggs', 'eggs', '</s>'): 0.08333333333333333, ('eggs', 'eggs', 'do'): 0.08333333333333333, ('eggs', 'eggs', 'not'): 0.08333333333333333, ('eggs', 'eggs', 'like'): 0.08333333333333333, ('eggs', 'eggs', 'green'): 0.08333333333333333, ('eggs', 'eggs', 'eggs'): 0.08333333333333333, ('eggs', 'eggs', 'and'): 0.08333333333333333, ('eggs', 'eggs', 'ham'): 0.08333333333333333, ('eggs', 'and', '<s>'): 0.07692307692307693, ('eggs', 'and', 'I'): 0.07692307692307693, ('eggs', 'and', 'am'): 0.07692307692307693, ('eggs', 'and', 'Sam'): 0.07692307692307693, ('eggs', 'and', '</s>'): 0.07692307692307693, ('eggs', 'and', 'do'): 0.07692307692307693, ('eggs', 'and', 'not'): 0.07692307692307693, ('eggs', 'and', 'like'): 0.07692307692307693, ('eggs', 'and', 'green'): 0.07692307692307693, ('eggs', 'and', 'eggs'): 0.07692307692307693, ('eggs', 'and', 'and'): 0.07692307692307693, ('eggs', 'ham', '<s>'): 0.08333333333333333, ('eggs', 'ham', 'I'): 0.08333333333333333, ('eggs', 'ham', 'am'): 0.08333333333333333, ('eggs', 'ham', 'Sam'): 0.08333333333333333, ('eggs', 'ham', '</s>'): 0.08333333333333333, ('eggs', 'ham', 'do'): 0.08333333333333333, ('eggs', 'ham', 'not'): 0.08333333333333333, ('eggs', 'ham', 'like'): 0.08333333333333333, ('eggs', 'ham', 'green'): 0.08333333333333333, ('eggs', 'ham', 'eggs'): 0.08333333333333333, ('eggs', 'ham', 'and'): 0.08333333333333333, ('eggs', 'ham', 'ham'): 0.08333333333333333, ('and', '<s>', '<s>'): 0.08333333333333333, ('and', '<s>', 'I'): 0.08333333333333333, ('and', '<s>', 'am'): 0.08333333333333333, ('and', '<s>', 'Sam'): 0.08333333333333333, ('and', '<s>', '</s>'): 0.08333333333333333, ('and', '<s>', 'do'): 0.08333333333333333, ('and', '<s>', 'not'): 0.08333333333333333, ('and', '<s>', 'like'): 0.08333333333333333, ('and', '<s>', 'green'): 0.08333333333333333, ('and', '<s>', 'eggs'): 0.08333333333333333, ('and', '<s>', 'and'): 0.08333333333333333, ('and', '<s>', 'ham'): 0.08333333333333333, ('and', 'I', '<s>'): 0.08333333333333333, ('and', 'I', 'I'): 0.08333333333333333, ('and', 'I', 'am'): 0.08333333333333333, ('and', 'I', 'Sam'): 0.08333333333333333, ('and', 'I', '</s>'): 0.08333333333333333, ('and', 'I', 'do'): 0.08333333333333333, ('and', 'I', 'not'): 0.08333333333333333, ('and', 'I', 'like'): 0.08333333333333333, ('and', 'I', 'green'): 0.08333333333333333, ('and', 'I', 'eggs'): 0.08333333333333333, ('and', 'I', 'and'): 0.08333333333333333, ('and', 'I', 'ham'): 0.08333333333333333, ('and', 'am', '<s>'): 0.08333333333333333, ('and', 'am', 'I'): 0.08333333333333333, ('and', 'am', 'am'): 0.08333333333333333, ('and', 'am', 'Sam'): 0.08333333333333333, ('and', 'am', '</s>'): 0.08333333333333333, ('and', 'am', 'do'): 0.08333333333333333, ('and', 'am', 'not'): 0.08333333333333333, ('and', 'am', 'like'): 0.08333333333333333, ('and', 'am', 'green'): 0.08333333333333333, ('and', 'am', 'eggs'): 0.08333333333333333, ('and', 'am', 'and'): 0.08333333333333333, ('and', 'am', 'ham'): 0.08333333333333333, ('and', 'Sam', '<s>'): 0.08333333333333333, ('and', 'Sam', 'I'): 0.08333333333333333, ('and', 'Sam', 'am'): 0.08333333333333333, ('and', 'Sam', 'Sam'): 0.08333333333333333, ('and', 'Sam', '</s>'): 0.08333333333333333, ('and', 'Sam', 'do'): 0.08333333333333333, ('and', 'Sam', 'not'): 0.08333333333333333, ('and', 'Sam', 'like'): 0.08333333333333333, ('and', 'Sam', 'green'): 0.08333333333333333, ('and', 'Sam', 'eggs'): 0.08333333333333333, ('and', 'Sam', 'and'): 0.08333333333333333, ('and', 'Sam', 'ham'): 0.08333333333333333, ('and', '</s>', '<s>'): 0.08333333333333333, ('and', '</s>', 'I'): 0.08333333333333333, ('and', '</s>', 'am'): 0.08333333333333333, ('and', '</s>', 'Sam'): 0.08333333333333333, ('and', '</s>', '</s>'): 0.08333333333333333, ('and', '</s>', 'do'): 0.08333333333333333, ('and', '</s>', 'not'): 0.08333333333333333, ('and', '</s>', 'like'): 0.08333333333333333, ('and', '</s>', 'green'): 0.08333333333333333, ('and', '</s>', 'eggs'): 0.08333333333333333, ('and', '</s>', 'and'): 0.08333333333333333, ('and', '</s>', 'ham'): 0.08333333333333333, ('and', 'do', '<s>'): 0.08333333333333333, ('and', 'do', 'I'): 0.08333333333333333, ('and', 'do', 'am'): 0.08333333333333333, ('and', 'do', 'Sam'): 0.08333333333333333, ('and', 'do', '</s>'): 0.08333333333333333, ('and', 'do', 'do'): 0.08333333333333333, ('and', 'do', 'not'): 0.08333333333333333, ('and', 'do', 'like'): 0.08333333333333333, ('and', 'do', 'green'): 0.08333333333333333, ('and', 'do', 'eggs'): 0.08333333333333333, ('and', 'do', 'and'): 0.08333333333333333, ('and', 'do', 'ham'): 0.08333333333333333, ('and', 'not', '<s>'): 0.08333333333333333, ('and', 'not', 'I'): 0.08333333333333333, ('and', 'not', 'am'): 0.08333333333333333, ('and', 'not', 'Sam'): 0.08333333333333333, ('and', 'not', '</s>'): 0.08333333333333333, ('and', 'not', 'do'): 0.08333333333333333, ('and', 'not', 'not'): 0.08333333333333333, ('and', 'not', 'like'): 0.08333333333333333, ('and', 'not', 'green'): 0.08333333333333333, ('and', 'not', 'eggs'): 0.08333333333333333, ('and', 'not', 'and'): 0.08333333333333333, ('and', 'not', 'ham'): 0.08333333333333333, ('and', 'like', '<s>'): 0.08333333333333333, ('and', 'like', 'I'): 0.08333333333333333, ('and', 'like', 'am'): 0.08333333333333333, ('and', 'like', 'Sam'): 0.08333333333333333, ('and', 'like', '</s>'): 0.08333333333333333, ('and', 'like', 'do'): 0.08333333333333333, ('and', 'like', 'not'): 0.08333333333333333, ('and', 'like', 'like'): 0.08333333333333333, ('and', 'like', 'green'): 0.08333333333333333, ('and', 'like', 'eggs'): 0.08333333333333333, ('and', 'like', 'and'): 0.08333333333333333, ('and', 'like', 'ham'): 0.08333333333333333, ('and', 'green', '<s>'): 0.08333333333333333, ('and', 'green', 'I'): 0.08333333333333333, ('and', 'green', 'am'): 0.08333333333333333, ('and', 'green', 'Sam'): 0.08333333333333333, ('and', 'green', '</s>'): 0.08333333333333333, ('and', 'green', 'do'): 0.08333333333333333, ('and', 'green', 'not'): 0.08333333333333333, ('and', 'green', 'like'): 0.08333333333333333, ('and', 'green', 'green'): 0.08333333333333333, ('and', 'green', 'eggs'): 0.08333333333333333, ('and', 'green', 'and'): 0.08333333333333333, ('and', 'green', 'ham'): 0.08333333333333333, ('and', 'eggs', '<s>'): 0.08333333333333333, ('and', 'eggs', 'I'): 0.08333333333333333, ('and', 'eggs', 'am'): 0.08333333333333333, ('and', 'eggs', 'Sam'): 0.08333333333333333, ('and', 'eggs', '</s>'): 0.08333333333333333, ('and', 'eggs', 'do'): 0.08333333333333333, ('and', 'eggs', 'not'): 0.08333333333333333, ('and', 'eggs', 'like'): 0.08333333333333333, ('and', 'eggs', 'green'): 0.08333333333333333, ('and', 'eggs', 'eggs'): 0.08333333333333333, ('and', 'eggs', 'and'): 0.08333333333333333, ('and', 'eggs', 'ham'): 0.08333333333333333, ('and', 'and', '<s>'): 0.08333333333333333, ('and', 'and', 'I'): 0.08333333333333333, ('and', 'and', 'am'): 0.08333333333333333, ('and', 'and', 'Sam'): 0.08333333333333333, ('and', 'and', '</s>'): 0.08333333333333333, ('and', 'and', 'do'): 0.08333333333333333, ('and', 'and', 'not'): 0.08333333333333333, ('and', 'and', 'like'): 0.08333333333333333, ('and', 'and', 'green'): 0.08333333333333333, ('and', 'and', 'eggs'): 0.08333333333333333, ('and', 'and', 'and'): 0.08333333333333333, ('and', 'and', 'ham'): 0.08333333333333333, ('and', 'ham', '<s>'): 0.07692307692307693, ('and', 'ham', 'I'): 0.07692307692307693, ('and', 'ham', 'am'): 0.07692307692307693, ('and', 'ham', 'Sam'): 0.07692307692307693, ('and', 'ham', 'do'): 0.07692307692307693, ('and', 'ham', 'not'): 0.07692307692307693, ('and', 'ham', 'like'): 0.07692307692307693, ('and', 'ham', 'green'): 0.07692307692307693, ('and', 'ham', 'eggs'): 0.07692307692307693, ('and', 'ham', 'and'): 0.07692307692307693, ('and', 'ham', 'ham'): 0.07692307692307693, ('ham', '<s>', '<s>'): 0.08333333333333333, ('ham', '<s>', 'I'): 0.08333333333333333, ('ham', '<s>', 'am'): 0.08333333333333333, ('ham', '<s>', 'Sam'): 0.08333333333333333, ('ham', '<s>', '</s>'): 0.08333333333333333, ('ham', '<s>', 'do'): 0.08333333333333333, ('ham', '<s>', 'not'): 0.08333333333333333, ('ham', '<s>', 'like'): 0.08333333333333333, ('ham', '<s>', 'green'): 0.08333333333333333, ('ham', '<s>', 'eggs'): 0.08333333333333333, ('ham', '<s>', 'and'): 0.08333333333333333, ('ham', '<s>', 'ham'): 0.08333333333333333, ('ham', 'I', '<s>'): 0.08333333333333333, ('ham', 'I', 'I'): 0.08333333333333333, ('ham', 'I', 'am'): 0.08333333333333333, ('ham', 'I', 'Sam'): 0.08333333333333333, ('ham', 'I', '</s>'): 0.08333333333333333, ('ham', 'I', 'do'): 0.08333333333333333, ('ham', 'I', 'not'): 0.08333333333333333, ('ham', 'I', 'like'): 0.08333333333333333, ('ham', 'I', 'green'): 0.08333333333333333, ('ham', 'I', 'eggs'): 0.08333333333333333, ('ham', 'I', 'and'): 0.08333333333333333, ('ham', 'I', 'ham'): 0.08333333333333333, ('ham', 'am', '<s>'): 0.08333333333333333, ('ham', 'am', 'I'): 0.08333333333333333, ('ham', 'am', 'am'): 0.08333333333333333, ('ham', 'am', 'Sam'): 0.08333333333333333, ('ham', 'am', '</s>'): 0.08333333333333333, ('ham', 'am', 'do'): 0.08333333333333333, ('ham', 'am', 'not'): 0.08333333333333333, ('ham', 'am', 'like'): 0.08333333333333333, ('ham', 'am', 'green'): 0.08333333333333333, ('ham', 'am', 'eggs'): 0.08333333333333333, ('ham', 'am', 'and'): 0.08333333333333333, ('ham', 'am', 'ham'): 0.08333333333333333, ('ham', 'Sam', '<s>'): 0.08333333333333333, ('ham', 'Sam', 'I'): 0.08333333333333333, ('ham', 'Sam', 'am'): 0.08333333333333333, ('ham', 'Sam', 'Sam'): 0.08333333333333333, ('ham', 'Sam', '</s>'): 0.08333333333333333, ('ham', 'Sam', 'do'): 0.08333333333333333, ('ham', 'Sam', 'not'): 0.08333333333333333, ('ham', 'Sam', 'like'): 0.08333333333333333, ('ham', 'Sam', 'green'): 0.08333333333333333, ('ham', 'Sam', 'eggs'): 0.08333333333333333, ('ham', 'Sam', 'and'): 0.08333333333333333, ('ham', 'Sam', 'ham'): 0.08333333333333333, ('ham', '</s>', '<s>'): 0.07692307692307693, ('ham', '</s>', 'I'): 0.07692307692307693, ('ham', '</s>', 'am'): 0.07692307692307693, ('ham', '</s>', 'Sam'): 0.07692307692307693, ('ham', '</s>', '</s>'): 0.07692307692307693, ('ham', '</s>', 'do'): 0.07692307692307693, ('ham', '</s>', 'not'): 0.07692307692307693, ('ham', '</s>', 'like'): 0.07692307692307693, ('ham', '</s>', 'green'): 0.07692307692307693, ('ham', '</s>', 'eggs'): 0.07692307692307693, ('ham', '</s>', 'and'): 0.07692307692307693, ('ham', '</s>', 'ham'): 0.07692307692307693, ('ham', 'do', '<s>'): 0.08333333333333333, ('ham', 'do', 'I'): 0.08333333333333333, ('ham', 'do', 'am'): 0.08333333333333333, ('ham', 'do', 'Sam'): 0.08333333333333333, ('ham', 'do', '</s>'): 0.08333333333333333, ('ham', 'do', 'do'): 0.08333333333333333, ('ham', 'do', 'not'): 0.08333333333333333, ('ham', 'do', 'like'): 0.08333333333333333, ('ham', 'do', 'green'): 0.08333333333333333, ('ham', 'do', 'eggs'): 0.08333333333333333, ('ham', 'do', 'and'): 0.08333333333333333, ('ham', 'do', 'ham'): 0.08333333333333333, ('ham', 'not', '<s>'): 0.08333333333333333, ('ham', 'not', 'I'): 0.08333333333333333, ('ham', 'not', 'am'): 0.08333333333333333, ('ham', 'not', 'Sam'): 0.08333333333333333, ('ham', 'not', '</s>'): 0.08333333333333333, ('ham', 'not', 'do'): 0.08333333333333333, ('ham', 'not', 'not'): 0.08333333333333333, ('ham', 'not', 'like'): 0.08333333333333333, ('ham', 'not', 'green'): 0.08333333333333333, ('ham', 'not', 'eggs'): 0.08333333333333333, ('ham', 'not', 'and'): 0.08333333333333333, ('ham', 'not', 'ham'): 0.08333333333333333, ('ham', 'like', '<s>'): 0.08333333333333333, ('ham', 'like', 'I'): 0.08333333333333333, ('ham', 'like', 'am'): 0.08333333333333333, ('ham', 'like', 'Sam'): 0.08333333333333333, ('ham', 'like', '</s>'): 0.08333333333333333, ('ham', 'like', 'do'): 0.08333333333333333, ('ham', 'like', 'not'): 0.08333333333333333, ('ham', 'like', 'like'): 0.08333333333333333, ('ham', 'like', 'green'): 0.08333333333333333, ('ham', 'like', 'eggs'): 0.08333333333333333, ('ham', 'like', 'and'): 0.08333333333333333, ('ham', 'like', 'ham'): 0.08333333333333333, ('ham', 'green', '<s>'): 0.08333333333333333, ('ham', 'green', 'I'): 0.08333333333333333, ('ham', 'green', 'am'): 0.08333333333333333, ('ham', 'green', 'Sam'): 0.08333333333333333, ('ham', 'green', '</s>'): 0.08333333333333333, ('ham', 'green', 'do'): 0.08333333333333333, ('ham', 'green', 'not'): 0.08333333333333333, ('ham', 'green', 'like'): 0.08333333333333333, ('ham', 'green', 'green'): 0.08333333333333333, ('ham', 'green', 'eggs'): 0.08333333333333333, ('ham', 'green', 'and'): 0.08333333333333333, ('ham', 'green', 'ham'): 0.08333333333333333, ('ham', 'eggs', '<s>'): 0.08333333333333333, ('ham', 'eggs', 'I'): 0.08333333333333333, ('ham', 'eggs', 'am'): 0.08333333333333333, ('ham', 'eggs', 'Sam'): 0.08333333333333333, ('ham', 'eggs', '</s>'): 0.08333333333333333, ('ham', 'eggs', 'do'): 0.08333333333333333, ('ham', 'eggs', 'not'): 0.08333333333333333, ('ham', 'eggs', 'like'): 0.08333333333333333, ('ham', 'eggs', 'green'): 0.08333333333333333, ('ham', 'eggs', 'eggs'): 0.08333333333333333, ('ham', 'eggs', 'and'): 0.08333333333333333, ('ham', 'eggs', 'ham'): 0.08333333333333333, ('ham', 'and', '<s>'): 0.08333333333333333, ('ham', 'and', 'I'): 0.08333333333333333, ('ham', 'and', 'am'): 0.08333333333333333, ('ham', 'and', 'Sam'): 0.08333333333333333, ('ham', 'and', '</s>'): 0.08333333333333333, ('ham', 'and', 'do'): 0.08333333333333333, ('ham', 'and', 'not'): 0.08333333333333333, ('ham', 'and', 'like'): 0.08333333333333333, ('ham', 'and', 'green'): 0.08333333333333333, ('ham', 'and', 'eggs'): 0.08333333333333333, ('ham', 'and', 'and'): 0.08333333333333333, ('ham', 'and', 'ham'): 0.08333333333333333, ('ham', 'ham', '<s>'): 0.08333333333333333, ('ham', 'ham', 'I'): 0.08333333333333333, ('ham', 'ham', 'am'): 0.08333333333333333, ('ham', 'ham', 'Sam'): 0.08333333333333333, ('ham', 'ham', '</s>'): 0.08333333333333333, ('ham', 'ham', 'do'): 0.08333333333333333, ('ham', 'ham', 'not'): 0.08333333333333333, ('ham', 'ham', 'like'): 0.08333333333333333, ('ham', 'ham', 'green'): 0.08333333333333333, ('ham', 'ham', 'eggs'): 0.08333333333333333, ('ham', 'ham', 'and'): 0.08333333333333333, ('ham', 'ham', 'ham'): 0.08333333333333333}

def test2():
    student_sm = count_based_ngram_with_smoothing(input1, 2)
    if not test_dict(smooth_out1, student_sm):
        return False
    student_sm = count_based_ngram_with_smoothing(input2, 3)
    if not test_dict(smooth_out2, student_sm):
        return False
    return True

sequence1 = ['like', 'green', 'eggs']
prob1 = 0.15384615384615385
prob2 = 1.0

sequence2 = ['not', 'like', 'do']
prob21  = 0.07692307692307693
prob22 = 0


def test3():
    if not approx_equal(prob_ngram(smooth_out2, sequence1, 3), prob1):
        return False
    if not approx_equal(prob_ngram(smooth_out2, sequence2, 3), prob21):
        return False
    return True


def test4():
    if not approx_equal(prob_ngram(out2, sequence1, 3), prob2):
        return False
    if not approx_equal(prob_ngram(out2, sequence2, 3), prob22):
        return False
    return True

def evaluate():
    score = 0 
    exercises = [(test1, 20), (test2, 40), (test3,20), (test4, 20)]
    for i, (test, sc) in enumerate(exercises):
        if test():
            print("Ex {}: {}/{}".format(i+1, sc, sc))
            score += sc
        else:
            print("Ex {}:  0/{}".format(i+1, sc))
            
    return score

print("Score: ", evaluate(), "/ 100")

Ex 1: 20/20
Ex 2: 40/40
Ex 3: 20/20
Ex 4: 20/20
Score:  100 / 100
